In [ ]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit, call_function
from snowflake.snowpark.types import *
import pandas as pd
import numpy as np
import time
import json
from typing import List, Dict, Tuple
from openai import OpenAI
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

In [ ]:
OPENAI_API_KEY = ""  # ⚠️ Replace with your actual OpenAI API key
SAMPLE_SIZE = 10000  # Fixed at 10,000 rows

# Initialize OpenAI client
if OPENAI_API_KEY == "YOUR_OPENAI_API_KEY_HERE":
    print("❌ ERROR: Please update OPENAI_API_KEY with your actual key!")
    print("   Get your key from: https://platform.openai.com/api-keys")
    client = None
else:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ OpenAI client initialized")
    print(f"📊 Sample size: {SAMPLE_SIZE:,} rows")

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Test session
print("✅ Snowflake session active")
print(f"   Current role: {session.get_current_role()}")
print(f"   Current database: {session.get_current_database()}")
print(f"   Current warehouse: {session.get_current_warehouse()}")

In [ ]:
source_table = "MEALMINDDATA.DBT_VAISHNAVIPATEL15_MARTS.MART_MEAL_PLANNING_CLEANED"
embeddings_schema = "MEALMINDDATA.EMBEDDINGS_COMPARISON"

# Create schema
session.sql(f"CREATE SCHEMA IF NOT EXISTS {embeddings_schema}").collect()
print(f"✅ Schema {embeddings_schema} ready")

# Check source table
row_count = session.sql(f"SELECT COUNT(*) as cnt FROM {source_table}").collect()[0]['CNT']
print(f"📊 Source table has {row_count:,} total rows")
print(f"📌 We will process {SAMPLE_SIZE:,} rows")


In [ ]:
source_table = "MEALMINDDATA.DBT_VAISHNAVIPATEL15_MARTS.MART_MEAL_PLANNING_CLEANED"
embeddings_schema = "MEALMINDDATA.EMBEDDINGS_COMPARISON"

In [ ]:
session.sql(f"""
    CREATE OR REPLACE TABLE {embeddings_schema}.SNOWFLAKE_EMBEDDINGS (
        FDC_ID NUMBER(38,0),
        FOOD_NAME VARCHAR(1000),
        CATEGORY VARCHAR(1000),
        EMBEDDING_TEXT VARCHAR,
        EMBEDDING VECTOR(FLOAT, 768),
        GENERATION_TIME_MS FLOAT,
        MODEL_NAME VARCHAR DEFAULT 'snowflake-arctic-embed-m',
        CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("✅ Created SNOWFLAKE_EMBEDDINGS table")

# OpenAI embeddings table
session.sql(f"""
    CREATE OR REPLACE TABLE {embeddings_schema}.OPENAI_EMBEDDINGS (
        FDC_ID NUMBER(38,0),
        FOOD_NAME VARCHAR(1000),
        CATEGORY VARCHAR(1000),
        EMBEDDING_TEXT VARCHAR,
        EMBEDDING VECTOR(FLOAT, 1536),
        GENERATION_TIME_MS FLOAT,
        MODEL_NAME VARCHAR DEFAULT 'text-embedding-3-small',
        CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("✅ Created OPENAI_EMBEDDINGS table")

# Metrics comparison table
session.sql(f"""
    CREATE OR REPLACE TABLE {embeddings_schema}.COMPARISON_METRICS (
        METRIC_TYPE VARCHAR,
        METRIC_NAME VARCHAR,
        SNOWFLAKE_VALUE VARCHAR,
        OPENAI_VALUE VARCHAR,
        COMPARISON VARCHAR,
        RUN_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

print("✅ Created COMPARISON_METRICS table")
print("\n📁 All tables created in schema:", embeddings_schema)


In [ ]:
session.sql(f"""
    CREATE OR REPLACE TABLE {embeddings_schema}.OPENAI_EMBEDDINGS (
        FDC_ID NUMBER(38,0),
        FOOD_NAME VARCHAR(1000),
        CATEGORY VARCHAR(1000),
        EMBEDDING_TEXT VARCHAR,
        EMBEDDING VECTOR(FLOAT, 1536),
        GENERATION_TIME_MS FLOAT,
        MODEL_NAME VARCHAR DEFAULT 'text-embedding-3-small',
        CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
""").collect()

In [ ]:
def prepare_data():
    query = f"""
        SELECT 
            FDC_ID,
            FOOD_NAME,
            CATEGORY,
            CALORIES,
            PROTEIN,
            CARBOHYDRATE,
            TOTAL_FAT,
            FIBER,
            SODIUM,
            CONCAT(
                'Food: ', FOOD_NAME, 
                ' Category: ', COALESCE(CATEGORY, 'Unknown'),
                ' Calories: ', COALESCE(CALORIES::VARCHAR, 'N/A'),
                ' Protein: ', COALESCE(PROTEIN::VARCHAR, 'N/A'), 'g',
                ' Carbs: ', COALESCE(CARBOHYDRATE::VARCHAR, 'N/A'), 'g',
                ' Fat: ', COALESCE(TOTAL_FAT::VARCHAR, 'N/A'), 'g',
                ' Fiber: ', COALESCE(FIBER::VARCHAR, 'N/A'), 'g',
                ' Sodium: ', COALESCE(SODIUM::VARCHAR, 'N/A'), 'mg'
            ) AS EMBEDDING_TEXT
        FROM {source_table}
        WHERE FOOD_NAME IS NOT NULL
        ORDER BY FDC_ID
        LIMIT {SAMPLE_SIZE}
    """
    return session.sql(query)

# Prepare data
data_df = prepare_data()
print(f"✅ Prepared {SAMPLE_SIZE:,} rows for embedding")

# Show sample
sample = data_df.limit(3).to_pandas()
print("\n📋 Sample data:")
for idx, row in sample.iterrows():
    print(f"\nFood: {row['FOOD_NAME']}")
    print(f"Text: {row['EMBEDDING_TEXT'][:100]}...")


In [ ]:
print("🔄 Generating Snowflake Arctic embeddings...")
print("   Processing first 5,000 rows...")

# Clear existing data
session.sql(f"TRUNCATE TABLE {embeddings_schema}.SNOWFLAKE_EMBEDDINGS").collect()

start_time = time.time()
batch_size = 1000

# Process first 5000 rows
for i in range(0, 5000, batch_size):
    print(f"   Batch {i//batch_size + 1}/5: rows {i+1}-{i+batch_size}")
    
    session.sql(f"""
        INSERT INTO {embeddings_schema}.SNOWFLAKE_EMBEDDINGS
        SELECT 
            FDC_ID,
            FOOD_NAME,
            CATEGORY,
            EMBEDDING_TEXT,
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', EMBEDDING_TEXT) as EMBEDDING,
            {(time.time() - start_time) * 1000 / (i + batch_size)} as GENERATION_TIME_MS,
            'snowflake-arctic-embed-m' as MODEL_NAME,
            CURRENT_TIMESTAMP() as CREATED_AT
        FROM (
            SELECT * FROM (
                SELECT 
                    FDC_ID,
                    FOOD_NAME,
                    CATEGORY,
                    CONCAT(
                        'Food: ', FOOD_NAME, 
                        ' Category: ', COALESCE(CATEGORY, 'Unknown'),
                        ' Calories: ', COALESCE(CALORIES::VARCHAR, 'N/A'),
                        ' Protein: ', COALESCE(PROTEIN::VARCHAR, 'N/A'), 'g',
                        ' Carbs: ', COALESCE(CARBOHYDRATE::VARCHAR, 'N/A'), 'g',
                        ' Fat: ', COALESCE(TOTAL_FAT::VARCHAR, 'N/A'), 'g'
                    ) AS EMBEDDING_TEXT
                FROM {source_table}
                WHERE FOOD_NAME IS NOT NULL
                ORDER BY FDC_ID
                LIMIT {i + batch_size}
            )
            MINUS
            SELECT * FROM (
                SELECT 
                    FDC_ID,
                    FOOD_NAME,
                    CATEGORY,
                    CONCAT(
                        'Food: ', FOOD_NAME, 
                        ' Category: ', COALESCE(CATEGORY, 'Unknown'),
                        ' Calories: ', COALESCE(CALORIES::VARCHAR, 'N/A'),
                        ' Protein: ', COALESCE(PROTEIN::VARCHAR, 'N/A'), 'g',
                        ' Carbs: ', COALESCE(CARBOHYDRATE::VARCHAR, 'N/A'), 'g',
                        ' Fat: ', COALESCE(TOTAL_FAT::VARCHAR, 'N/A'), 'g'
                    ) AS EMBEDDING_TEXT
                FROM {source_table}
                WHERE FOOD_NAME IS NOT NULL
                ORDER BY FDC_ID
                LIMIT {i}
            )
        )
    """).collect()

count = session.table(f"{embeddings_schema}.SNOWFLAKE_EMBEDDINGS").count()
print(f"✅ Generated {count:,} embeddings so far")

In [ ]:
print("   Processing second 5,000 rows...")

# Process second 5000 rows
for i in range(5000, 10000, batch_size):
    print(f"   Batch {i//batch_size + 1}/10: rows {i+1}-{i+batch_size}")
    
    session.sql(f"""
        INSERT INTO {embeddings_schema}.SNOWFLAKE_EMBEDDINGS
        SELECT 
            FDC_ID,
            FOOD_NAME,
            CATEGORY,
            EMBEDDING_TEXT,
            SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', EMBEDDING_TEXT) as EMBEDDING,
            {(time.time() - start_time) * 1000 / (i + batch_size)} as GENERATION_TIME_MS,
            'snowflake-arctic-embed-m' as MODEL_NAME,
            CURRENT_TIMESTAMP() as CREATED_AT
        FROM (
            SELECT * FROM (
                SELECT 
                    FDC_ID,
                    FOOD_NAME,
                    CATEGORY,
                    CONCAT(
                        'Food: ', FOOD_NAME, 
                        ' Category: ', COALESCE(CATEGORY, 'Unknown'),
                        ' Calories: ', COALESCE(CALORIES::VARCHAR, 'N/A'),
                        ' Protein: ', COALESCE(PROTEIN::VARCHAR, 'N/A'), 'g',
                        ' Carbs: ', COALESCE(CARBOHYDRATE::VARCHAR, 'N/A'), 'g',
                        ' Fat: ', COALESCE(TOTAL_FAT::VARCHAR, 'N/A'), 'g'
                    ) AS EMBEDDING_TEXT
                FROM {source_table}
                WHERE FOOD_NAME IS NOT NULL
                ORDER BY FDC_ID
                LIMIT {i + batch_size}
            )
            MINUS
            SELECT * FROM (
                SELECT 
                    FDC_ID,
                    FOOD_NAME,
                    CATEGORY,
                    CONCAT(
                        'Food: ', FOOD_NAME, 
                        ' Category: ', COALESCE(CATEGORY, 'Unknown'),
                        ' Calories: ', COALESCE(CALORIES::VARCHAR, 'N/A'),
                        ' Protein: ', COALESCE(PROTEIN::VARCHAR, 'N/A'), 'g',
                        ' Carbs: ', COALESCE(CARBOHYDRATE::VARCHAR, 'N/A'), 'g',
                        ' Fat: ', COALESCE(TOTAL_FAT::VARCHAR, 'N/A'), 'g'
                    ) AS EMBEDDING_TEXT
                FROM {source_table}
                WHERE FOOD_NAME IS NOT NULL
                ORDER BY FDC_ID
                LIMIT {i}
            )
        )
    """).collect()

total_time = (time.time() - start_time) * 1000
final_count = session.table(f"{embeddings_schema}.SNOWFLAKE_EMBEDDINGS").count()

print(f"\n✅ Snowflake Arctic Embedding Generation Complete!")
print(f"   Total embeddings: {final_count:,}")
print(f"   Total time: {total_time/1000:.2f} seconds")
print(f"   Average time per embedding: {total_time/final_count:.2f}ms")


In [ ]:
def get_openai_embeddings_batch(texts: List[str], client: OpenAI) -> Tuple[List[List[float]], int]:
    """Get embeddings for a batch of texts using OpenAI API"""
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=texts
        )
        
        # Extract embeddings and maintain order
        embeddings = []
        for item in response.data:
            embeddings.append(item.embedding)
        
        # Get token usage
        total_tokens = response.usage.total_tokens if hasattr(response, 'usage') else len(texts) * 50  # estimate
        
        return embeddings, total_tokens
    except Exception as e:
        print(f"Error getting embeddings: {e}")
        return None, 0

print("✅ OpenAI helper function ready")

In [ ]:
print("🔄 Generating OpenAI embeddings...")
print("   This will take 15-20 minutes due to API rate limits")

if client is None:
    print("❌ Skipping: OpenAI API key not configured")
else:
    # Clear existing data
    session.sql(f"TRUNCATE TABLE {embeddings_schema}.OPENAI_EMBEDDINGS").collect()
    
    # Get data as pandas DataFrame
    print("   Loading data into memory...")
    data_pandas = data_df.to_pandas()
    
    batch_size = 100  # OpenAI can handle larger batches
    total_rows = min(len(data_pandas), SAMPLE_SIZE)
    total_batches = (total_rows + batch_size - 1) // batch_size
    
    start_time = time.time()
    total_tokens = 0
    successful_embeddings = 0
    
    print(f"   Processing {total_rows:,} rows in {total_batches} batches")
    print("   Processing first 5,000 rows...")
    
    # Process first 5000 rows
    embedding_records = []
    
    for batch_idx in range(0, min(5000, total_rows), batch_size):
        batch_end = min(batch_idx + batch_size, min(5000, total_rows))
        batch = data_pandas.iloc[batch_idx:batch_end]
        current_batch = batch_idx // batch_size + 1
        
        if current_batch % 5 == 0:  # Progress update every 5 batches
            elapsed = time.time() - start_time
            rate = successful_embeddings / elapsed if elapsed > 0 else 0
            print(f"   Batch {current_batch}/{min(50, total_batches)}: {successful_embeddings:,} embeddings "
                  f"({rate:.1f}/sec), {total_tokens:,} tokens used")
        
        # Get embeddings for batch
        texts = batch['EMBEDDING_TEXT'].tolist()
        embeddings, tokens = get_openai_embeddings_batch(texts, client)
        
        if embeddings:
            total_tokens += tokens
            
            # Prepare records for insertion
            for idx, (_, row) in enumerate(batch.iterrows()):
                if idx < len(embeddings) and embeddings[idx]:
                    embedding_records.append({
                        'FDC_ID': int(row['FDC_ID']),
                        'FOOD_NAME': row['FOOD_NAME'],
                        'CATEGORY': row['CATEGORY'] if pd.notna(row['CATEGORY']) else None,
                        'EMBEDDING_TEXT': row['EMBEDDING_TEXT'],
                        'EMBEDDING': embeddings[idx],
                        'GENERATION_TIME_MS': (time.time() - start_time) * 1000 / (successful_embeddings + 1)
                    })
                    successful_embeddings += 1
        
        # Add delay to respect rate limits
        time.sleep(0.5)
    
    print(f"   First batch complete: {len(embedding_records):,} embeddings ready to insert")


In [ ]:
if client is not None and total_rows > 5000:
    print("   Processing second 5,000 rows...")
    
    # Process second 5000 rows
    for batch_idx in range(5000, min(10000, total_rows), batch_size):
        batch_end = min(batch_idx + batch_size, min(10000, total_rows))
        batch = data_pandas.iloc[batch_idx:batch_end]
        current_batch = batch_idx // batch_size + 1
        
        if current_batch % 5 == 0:  # Progress update
            elapsed = time.time() - start_time
            rate = successful_embeddings / elapsed if elapsed > 0 else 0
            print(f"   Batch {current_batch}/{total_batches}: {successful_embeddings:,} embeddings "
                  f"({rate:.1f}/sec), {total_tokens:,} tokens used")
        
        # Get embeddings for batch
        texts = batch['EMBEDDING_TEXT'].tolist()
        embeddings, tokens = get_openai_embeddings_batch(texts, client)
        
        if embeddings:
            total_tokens += tokens
            
            # Prepare records for insertion
            for idx, (_, row) in enumerate(batch.iterrows()):
                if idx < len(embeddings) and embeddings[idx]:
                    embedding_records.append({
                        'FDC_ID': int(row['FDC_ID']),
                        'FOOD_NAME': row['FOOD_NAME'],
                        'CATEGORY': row['CATEGORY'] if pd.notna(row['CATEGORY']) else None,
                        'EMBEDDING_TEXT': row['EMBEDDING_TEXT'],
                        'EMBEDDING': embeddings[idx],
                        'GENERATION_TIME_MS': (time.time() - start_time) * 1000 / (successful_embeddings + 1)
                    })
                    successful_embeddings += 1
        
        # Add delay to respect rate limits
        time.sleep(0.5)
    
    # Insert all embeddings to Snowflake
    if embedding_records:
        print(f"\n📤 Inserting {len(embedding_records):,} embeddings to Snowflake...")
        
        # Convert to DataFrame
        embeddings_df = pd.DataFrame(embedding_records)
        
        # Create schema for Snowpark DataFrame
        schema = StructType([
            StructField("FDC_ID", LongType()),
            StructField("FOOD_NAME", StringType()),
            StructField("CATEGORY", StringType()),
            StructField("EMBEDDING_TEXT", StringType()),
            StructField("EMBEDDING", ArrayType(FloatType())),
            StructField("GENERATION_TIME_MS", FloatType())
        ])
        
        # Create Snowpark DataFrame
        snow_df = session.create_dataframe(embeddings_df, schema=schema)
        
        # Create temporary table
        snow_df.create_or_replace_temp_view("temp_openai_embeddings")
        
        # Insert with vector conversion
        insert_count = session.sql(f"""
            INSERT INTO {embeddings_schema}.OPENAI_EMBEDDINGS
            SELECT 
                FDC_ID,
                FOOD_NAME,
                CATEGORY,
                EMBEDDING_TEXT,
                EMBEDDING::VECTOR(FLOAT, 1536) as EMBEDDING,
                GENERATION_TIME_MS,
                'text-embedding-3-small' as MODEL_NAME,
                CURRENT_TIMESTAMP() as CREATED_AT
            FROM temp_openai_embeddings
        """).collect()
        
        print(f"✅ Successfully inserted embeddings to table")
    
    total_time = (time.time() - start_time)
    print(f"\n✅ OpenAI Embedding Generation Complete!")
    print(f"   Total embeddings: {successful_embeddings:,}")
    print(f"   Total time: {total_time:.2f} seconds")
    print(f"   Average time: {total_time/successful_embeddings*1000:.2f}ms per embedding")
    print(f"   Total tokens used: {total_tokens:,}")
    print(f"   Estimated cost: ${total_tokens * 0.00002:.4f} (@$0.020 per 1M tokens)")


In [ ]:
print("📊 Calculating Performance Metrics...")

# Snowflake metrics
snowflake_metrics = session.sql(f"""
    SELECT 
        COUNT(*) as count,
        AVG(GENERATION_TIME_MS) as avg_time,
        MIN(GENERATION_TIME_MS) as min_time,
        MAX(GENERATION_TIME_MS) as max_time,
        STDDEV(GENERATION_TIME_MS) as std_time
    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS
""").to_pandas()

print("\n📍 Snowflake Arctic Performance:")
print(f"   Count: {snowflake_metrics['COUNT'][0]:,}")
print(f"   Avg time: {snowflake_metrics['AVG_TIME'][0]:.2f}ms")
print(f"   Min/Max: {snowflake_metrics['MIN_TIME'][0]:.2f}ms / {snowflake_metrics['MAX_TIME'][0]:.2f}ms")

# OpenAI metrics
openai_metrics = session.sql(f"""
    SELECT 
        COUNT(*) as count,
        AVG(GENERATION_TIME_MS) as avg_time,
        MIN(GENERATION_TIME_MS) as min_time,
        MAX(GENERATION_TIME_MS) as max_time,
        STDDEV(GENERATION_TIME_MS) as std_time
    FROM {embeddings_schema}.OPENAI_EMBEDDINGS
""").to_pandas()

print("\n📍 OpenAI text-embedding-3-small Performance:")
print(f"   Count: {openai_metrics['COUNT'][0]:,}")
if openai_metrics['COUNT'][0] > 0:
    print(f"   Avg time: {openai_metrics['AVG_TIME'][0]:.2f}ms")
    print(f"   Min/Max: {openai_metrics['MIN_TIME'][0]:.2f}ms / {openai_metrics['MAX_TIME'][0]:.2f}ms")
    
    # Speed comparison
    if snowflake_metrics['AVG_TIME'][0] > 0 and openai_metrics['AVG_TIME'][0] > 0:
        speed_ratio = openai_metrics['AVG_TIME'][0] / snowflake_metrics['AVG_TIME'][0]
        print(f"\n⚡ Snowflake is {speed_ratio:.2f}x faster than OpenAI")

In [ ]:
print("💾 Storage Comparison...")

snowflake_count = snowflake_metrics['COUNT'][0]
openai_count = openai_metrics['COUNT'][0]

snowflake_storage_mb = snowflake_count * 768 * 4 / (1024 * 1024)
openai_storage_mb = openai_count * 1536 * 4 / (1024 * 1024)

print(f"\n📍 Snowflake Arctic:")
print(f"   Dimensions: 768")
print(f"   Storage: {snowflake_storage_mb:.2f} MB for {snowflake_count:,} embeddings")

print(f"\n📍 OpenAI:")
print(f"   Dimensions: 1536")
print(f"   Storage: {openai_storage_mb:.2f} MB for {openai_count:,} embeddings")

if snowflake_storage_mb > 0 and openai_storage_mb > 0:
    # Calculate per-embedding storage
    snowflake_per_embedding = snowflake_storage_mb / snowflake_count * 1024  # in KB
    openai_per_embedding = openai_storage_mb / openai_count * 1024  # in KB
    
    print(f"\n📊 Per-embedding storage:")
    print(f"   Snowflake: {snowflake_per_embedding:.2f} KB")
    print(f"   OpenAI: {openai_per_embedding:.2f} KB")
    
    # If both have same count, calculate savings
    if snowflake_count == openai_count:
        savings = openai_storage_mb - snowflake_storage_mb
        savings_pct = (1 - snowflake_storage_mb/openai_storage_mb) * 100
        print(f"\n💰 Storage Savings with Snowflake: {savings:.2f} MB ({savings_pct:.1f}%)")


In [ ]:
def search_snowflake(query: str, top_k: int = 5):
    """Semantic search using Snowflake embeddings"""
    try:
        results = session.sql(f"""
            WITH query_embed AS (
                SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                    'snowflake-arctic-embed-m', 
                    '{query.replace("'", "''")}' 
                ) as QUERY_EMBEDDING
            )
            SELECT 
                s.FOOD_NAME,
                s.CATEGORY,
                VECTOR_COSINE_SIMILARITY(
                    s.EMBEDDING, 
                    q.QUERY_EMBEDDING
                ) as SIMILARITY
            FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s,
                 query_embed q
            ORDER BY SIMILARITY DESC
            LIMIT {top_k}
        """).to_pandas()
        return results
    except Exception as e:
        print(f"Error in Snowflake search: {e}")
        return pd.DataFrame()

def search_openai(query: str, top_k: int = 5):
    """Semantic search using OpenAI embeddings"""
    if client is None:
        print("OpenAI search skipped - no API key")
        return pd.DataFrame()
    
    try:
        # Get query embedding
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=query
        )
        query_embedding = response.data[0].embedding
        
        # Convert to string format for SQL
        embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
        
        results = session.sql(f"""
            SELECT 
                FOOD_NAME,
                CATEGORY,
                VECTOR_COSINE_SIMILARITY(
                    EMBEDDING,
                    {embedding_str}::VECTOR(FLOAT, 1536)
                ) as SIMILARITY
            FROM {embeddings_schema}.OPENAI_EMBEDDINGS
            ORDER BY SIMILARITY DESC
            LIMIT {top_k}
        """).to_pandas()
        return results
    except Exception as e:
        print(f"Error in OpenAI search: {e}")
        return pd.DataFrame()

print("✅ Search functions defined")

In [ ]:
test_queries = [
    "healthy vegetarian high protein meals",
    "low calorie breakfast options",
    "quick snacks under 200 calories",
    "mediterranean diet lunch",
    "post workout recovery food"
]

print("🔍 Running Semantic Search Comparison...")
print("="*60)

overlap_scores = []

for query in test_queries[:3]:  # Test first 3 queries
    print(f"\n📝 Query: '{query}'")
    print("-"*50)
    
    # Time Snowflake search
    start = time.time()
    snowflake_results = search_snowflake(query)
    snowflake_time = (time.time() - start) * 1000
    
    if not snowflake_results.empty:
        print(f"\n📍 Snowflake Arctic Results (Top 5) - {snowflake_time:.2f}ms:")
        for idx, row in snowflake_results.iterrows():
            print(f"   {idx+1}. {row['FOOD_NAME'][:40]:<40} Score: {row['SIMILARITY']:.4f}")
    
    # Time OpenAI search
    start = time.time()
    openai_results = search_openai(query)
    openai_time = (time.time() - start) * 1000
    
    if not openai_results.empty:
        print(f"\n📍 OpenAI Results (Top 5) - {openai_time:.2f}ms:")
        for idx, row in openai_results.iterrows():
            print(f"   {idx+1}. {row['FOOD_NAME'][:40]:<40} Score: {row['SIMILARITY']:.4f}")
    
    # Calculate overlap
    if not snowflake_results.empty and not openai_results.empty:
        snow_foods = set(snowflake_results['FOOD_NAME'].tolist())
        openai_foods = set(openai_results['FOOD_NAME'].tolist())
        overlap = len(snow_foods.intersection(openai_foods))
        overlap_pct = overlap / 5 * 100
        overlap_scores.append(overlap_pct)
        
        print(f"\n📊 Result Analysis:")
        print(f"   • Common foods: {overlap}/5 ({overlap_pct:.0f}% overlap)")
        print(f"   • Unique to Snowflake: {5-overlap}")
        print(f"   • Unique to OpenAI: {5-overlap}")
        print(f"   • Search times: Snowflake {snowflake_time:.2f}ms vs OpenAI {openai_time:.2f}ms")

if overlap_scores:
    print(f"\n📈 Average overlap across all queries: {np.mean(overlap_scores):.1f}%")


In [ ]:
print("\n" + "="*70)
print("📈 FINAL COMPARISON REPORT - 10,000 EMBEDDINGS")
print("="*70)

print("\n1️⃣ GENERATION PERFORMANCE")
print("-"*50)
if snowflake_metrics['COUNT'][0] > 0:
    print(f"Snowflake Arctic:")
    print(f"  • Embeddings generated: {snowflake_metrics['COUNT'][0]:,}")
    print(f"  • Average generation time: {snowflake_metrics['AVG_TIME'][0]:.2f}ms")
    print(f"  • Total storage: {snowflake_storage_mb:.2f} MB")
    print(f"  • Cost: Included in Snowflake compute")

if openai_metrics['COUNT'][0] > 0:
    print(f"\nOpenAI text-embedding-3-small:")
    print(f"  • Embeddings generated: {openai_metrics['COUNT'][0]:,}")
    print(f"  • Average generation time: {openai_metrics['AVG_TIME'][0]:.2f}ms")
    print(f"  • Total storage: {openai_storage_mb:.2f} MB")
    if 'total_tokens' in locals():
        print(f"  • API cost: ${total_tokens * 0.00002:.4f} for {total_tokens:,} tokens")

print("\n2️⃣ KEY PERFORMANCE INDICATORS")
print("-"*50)
if snowflake_metrics['COUNT'][0] > 0 and openai_metrics['COUNT'][0] > 0:
    speed_advantage = openai_metrics['AVG_TIME'][0] / snowflake_metrics['AVG_TIME'][0]
    storage_advantage = 2.0  # OpenAI uses 2x dimensions
    
    print(f"⚡ Speed: Snowflake is {speed_advantage:.1f}x faster")
    print(f"💾 Storage: Snowflake uses {1/storage_advantage:.1%} of OpenAI's storage")
    print(f"💰 Cost: Snowflake has no API costs vs OpenAI's per-token pricing")
    if overlap_scores:
        print(f"🎯 Accuracy: {np.mean(overlap_scores):.0f}% average result overlap")

print("\n3️⃣ QUALITATIVE COMPARISON")
print("-"*50)
print("✅ Snowflake Arctic Advantages:")
print("  • Blazing fast generation (native integration)")
print("  • 50% less storage required")
print("  • No API key management")
print("  • No rate limits or quotas")
print("  • Included in compute costs")

print("\n✅ OpenAI Advantages:")
print("  • Higher dimensional embeddings (1536 vs 768)")
print("  • May capture more subtle semantic nuances")
print("  • Trained on broader internet corpus")
print("  • Better for cross-domain applications")

print("\n4️⃣ RECOMMENDATIONS BY USE CASE")
print("-"*50)
print("📌 Choose Snowflake Arctic for:")
print("  • Production workloads at scale")
print("  • Real-time embedding generation")
print("  • Cost-sensitive deployments")
print("  • Applications staying within Snowflake")
print("  • Batch processing of large datasets")

print("\n📌 Choose OpenAI for:")
print("  • Research requiring maximum accuracy")
print("  • Cross-domain semantic understanding")
print("  • Applications already using OpenAI APIs")
print("  • Small-scale prototypes (<1000 docs)")


In [ ]:
WITH query AS (
    SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
        'snowflake-arctic-embed-m', 
        'high carb lunch'
    ) as embedding
)
SELECT 
    FOOD_NAME,
    CATEGORY,
    VECTOR_COSINE_SIMILARITY(EMBEDDING, (SELECT embedding FROM query)) as SCORE
FROM MEALMINDDATA.EMBEDDINGS_COMPARISON.SNOWFLAKE_EMBEDDINGS
ORDER BY SCORE DESC
LIMIT 10;

In [ ]:
# RAG with Snowflake Cortex LLM - Complete Pipeline
def rag_query_with_llm(query: str, top_k: int = 5):
    """
    Complete RAG pipeline using Snowflake only:
    1. Retrieve relevant context using Snowflake Arctic embeddings
    2. Generate answer using Snowflake Cortex Complete (Mistral)
    
    Args:
        query: User question
        top_k: Number of documents to retrieve
    """
    print(f"🔍 RAG Query: '{query}'")
    print("="*70)
    
    # Step 1: Retrieve relevant context using Snowflake embeddings
    print("\n📚 Step 1: Retrieving relevant context...")
    results = search_snowflake(query, top_k)
    
    if results.empty:
        print("❌ No results found")
        return None
    
    print(f"✅ Retrieved {len(results)} documents using Snowflake Arctic")
    for idx, row in results.iterrows():
        print(f"  {idx+1}. {row['FOOD_NAME']} (Score: {row['SIMILARITY']:.4f})")
    
    # Step 2: Build context from retrieved documents
    context = "\n".join([
        f"- {row['FOOD_NAME']} (Category: {row['CATEGORY']}, Relevance Score: {row['SIMILARITY']:.4f})"
        for _, row in results.iterrows()
    ])
    
    # Step 3: Generate answer using Snowflake Cortex LLM
    print("\n🤖 Step 2: Generating answer with Snowflake Cortex LLM...")
    
    prompt = f"""Based on the following food items from our database, answer the user's question.

Context (Relevant Foods):
{context}

User Question: {query}

Provide a helpful, create a meal that recommends specific foods from the context above. Include the food names and explain why they match the query."""

    try:
        # Using Snowflake Cortex Complete function with Mistral-Large2
        llm_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{prompt.replace("'", "''")}'
        ) as RESPONSE
        """
        
        response = session.sql(llm_query).collect()[0]['RESPONSE']
        
        print("\n💬 LLM Response:")
        print("-"*70)
        print(response)
        print("-"*70)
        
        return {
            'query': query,
            'retrieved_foods': results['FOOD_NAME'].tolist(),
            'similarity_scores': results['SIMILARITY'].tolist(),
            'llm_response': response,
            'embedding_model': 'snowflake-arctic-embed-m',
            'llm_model': 'mistral-large2'
        }
        
    except Exception as e:
        print(f"❌ Error generating LLM response: {e}")
        return None


# Batch RAG queries for testing
def batch_rag_queries(queries: list, top_k: int = 5):
    """
    Run multiple RAG queries and collect results
    """
    results = []
    
    print("🧪 Testing RAG System with Multiple Queries")
    print("="*70)
    
    for i, query in enumerate(queries, 1):
        print(f"\n\n{'='*70}")
        print(f"Query {i}/{len(queries)}")
        print("="*70)
        
        result = rag_query_with_llm(query, top_k)
        
        if result:
            results.append(result)
            print(f"\n✅ Successfully answered: '{query}'")
        
        time.sleep(1)  # Brief pause between queries
    
    return results


print("✅ RAG with Snowflake Cortex LLM functions ready!")
print("\nExample usage:")
print("  result = rag_query_with_llm('What are good high protein breakfast options?')")
print("  results = batch_rag_queries(['healthy snacks', 'low calorie lunch', 'post workout food'])")


In [ ]:
# Test RAG with various nutrition queries
test_queries = [
    "What are good high protein breakfast options?",
    "Suggest healthy low calorie snacks under 200 calories",
    "What foods are good for post-workout recovery?",
    "Mediterranean diet lunch ideas"
]

# Run single query
print("🧪 Single Query Test")
print("="*70)
result = rag_query_with_llm(test_queries[1], top_k=5)

if result:
    print(f"\n📊 Query Results Summary:")
    print(f"  Retrieved {len(result['retrieved_foods'])} relevant foods")
    print(f"  Using: {result['embedding_model']} + {result['llm_model']}")


In [ ]:
# RAG with Meal Plan Generation - FIXED VERSION
def rag_generate_meal_plan(
    days: int = 2,
    calorie_limit: int = 2000,
    carb_limit: int = 200,
    protein_target: int = None,
    dietary_preferences: str = None,
    top_k: int = 30
):
    """
    Generate a multi-day meal plan using RAG with Snowflake Cortex.
    
    Args:
        days: Number of days for meal plan (default: 2)
        calorie_limit: Daily calorie limit
        carb_limit: Daily carb limit in grams
        protein_target: Target protein in grams (optional)
        dietary_preferences: e.g., "vegetarian", "high-protein", "low-fat"
        top_k: Number of foods to retrieve from database
    """
    print(f"🍽️ Generating {days}-Day Meal Plan")
    print("="*70)
    print(f"📊 Constraints: {calorie_limit} cal/day, {carb_limit}g carbs/day")
    if protein_target:
        print(f"🥩 Protein target: {protein_target}g/day")
    if dietary_preferences:
        print(f"🥗 Preferences: {dietary_preferences}")
    print("="*70)
    
    # Build search query based on preferences
    if dietary_preferences:
        search_query = f"{dietary_preferences} meals breakfast lunch dinner snacks"
    else:
        search_query = "healthy balanced meals breakfast lunch dinner snacks"
    
    # Step 1: Retrieve relevant foods from database
    print(f"\n📚 Step 1: Retrieving relevant foods from database...")
    results = search_snowflake(search_query, top_k=top_k)
    
    if results.empty:
        print("❌ No results found")
        return None
    
    print(f"✅ Retrieved {len(results)} relevant foods")
    
    # Step 2: Get food names and fetch detailed nutritional info
    print("\n📊 Step 2: Fetching detailed nutritional information...")
    
    food_names = results['FOOD_NAME'].tolist()
    
    # Build query to get nutrition details for these foods
    food_names_escaped = ["'" + name.replace("'", "''") + "'" for name in food_names]
    food_names_str = ','.join(food_names_escaped)
    
    nutrition_query = f"""
    SELECT 
        FOOD_NAME,
        CATEGORY,
        COALESCE(CALORIES, 0) as CALORIES,
        COALESCE(PROTEIN, 0) as PROTEIN,
        COALESCE(CARBOHYDRATE, 0) as CARBS,
        COALESCE(TOTAL_FAT, 0) as FAT,
        COALESCE(FIBER, 0) as FIBER
    FROM {source_table}
    WHERE FOOD_NAME IN ({food_names_str})
    LIMIT {top_k}
    """
    
    try:
        nutrition_data = session.sql(nutrition_query).to_pandas()
    except Exception as e:
        print(f"⚠️ Could not fetch detailed nutrition. Using search results only.")
        nutrition_data = results
    
    # Build detailed context for LLM
    food_context = []
    for _, row in nutrition_data.iterrows():
        food_name = row['FOOD_NAME']
        category = row.get('CATEGORY', 'Unknown')
        calories = row.get('CALORIES', 0)
        protein = row.get('PROTEIN', 0)
        carbs = row.get('CARBS', 0) if 'CARBS' in row else row.get('CARBOHYDRATE', 0)
        fat = row.get('FAT', 0) if 'FAT' in row else row.get('TOTAL_FAT', 0)
        fiber = row.get('FIBER', 0)
        
        food_context.append(
            f"- {food_name} (Category: {category}): "
            f"{calories} cal, {protein}g protein, "
            f"{carbs}g carbs, {fat}g fat, {fiber}g fiber"
        )
    
    context = "\n".join(food_context)
    
    # Step 3: Build comprehensive prompt for meal plan generation
    print("\n🤖 Step 3: Generating meal plan with LLM...")
    
    prompt = f"""You are a professional nutritionist creating a detailed meal plan. 

AVAILABLE FOODS FROM DATABASE:
{context}

REQUIREMENTS:
- Create a {days}-day meal plan
- Daily calorie limit: {calorie_limit} calories
- Daily carb limit: {carb_limit} grams
{f'- Daily protein target: {protein_target} grams' if protein_target else ''}
{f'- Dietary preference: {dietary_preferences}' if dietary_preferences else ''}
- Must include: Breakfast, Lunch, Snacks, Dinner for each day
- ONLY use foods from the available database list above
- Stay within the daily limits

OUTPUT FORMAT:
For each day, provide:

**Day [X]:**

**Breakfast:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Lunch:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Snacks:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Dinner:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Day [X] Totals:**
- Total Calories: [X] / {calorie_limit} cal
- Total Protein: [X]g
- Total Carbs: [X] / {carb_limit}g
- Total Fat: [X]g
- Status: [Within/Over] limits

IMPORTANT RULES:
1. Select appropriate portion sizes to meet but not exceed limits
2. Distribute calories reasonably across meals (breakfast: ~25%, lunch: ~30%, snacks: ~15%, dinner: ~30%)
3. Calculate exact totals for each day
4. Verify daily totals are within constraints
5. Use realistic portion sizes (e.g., "1 cup", "100g", "1 medium", "2 pieces")
6. Choose foods that complement each other for balanced nutrition
7. Only use foods explicitly listed in the available database

Generate the complete {days}-day meal plan now:"""

    try:
        # Call Snowflake Cortex Complete
        llm_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{prompt.replace("'", "''")}'
        ) as MEAL_PLAN
        """
        
        response = session.sql(llm_query).collect()[0]['MEAL_PLAN']
        
        print("\n" + "="*70)
        print(f"📋 {days}-DAY MEAL PLAN")
        print("="*70)
        print(response)
        print("="*70)
        
        return {
            'days': days,
            'calorie_limit': calorie_limit,
            'carb_limit': carb_limit,
            'protein_target': protein_target,
            'dietary_preferences': dietary_preferences,
            'available_foods_count': len(nutrition_data),
            'meal_plan': response,
            'nutrition_data': nutrition_data
        }
        
    except Exception as e:
        print(f"❌ Error generating meal plan: {e}")
        return None


# Convenience function with common presets
def quick_meal_plan(preset: str = "balanced"):
    """
    Generate meal plans with common presets
    
    Presets:
    - "balanced": 2000 cal, 200g carbs
    - "low_carb": 1800 cal, 100g carbs, high protein
    - "high_protein": 2200 cal, 180g carbs, 150g protein
    - "weight_loss": 1500 cal, 150g carbs
    - "vegetarian": 2000 cal, 220g carbs
    """
    presets = {
        "balanced": {
            "days": 2,
            "calorie_limit": 2000,
            "carb_limit": 200,
            "protein_target": 100,
            "dietary_preferences": None
        },
        "low_carb": {
            "days": 2,
            "calorie_limit": 1800,
            "carb_limit": 100,
            "protein_target": 120,
            "dietary_preferences": "high protein low carb"
        },
        "high_protein": {
            "days": 2,
            "calorie_limit": 2200,
            "carb_limit": 180,
            "protein_target": 150,
            "dietary_preferences": "high protein"
        },
        "weight_loss": {
            "days": 2,
            "calorie_limit": 1500,
            "carb_limit": 150,
            "protein_target": 90,
            "dietary_preferences": "low calorie healthy"
        },
        "vegetarian": {
            "days": 2,
            "calorie_limit": 2000,
            "carb_limit": 220,
            "protein_target": 80,
            "dietary_preferences": "vegetarian plant-based"
        }
    }
    
    if preset not in presets:
        print(f"❌ Unknown preset. Available: {list(presets.keys())}")
        return None
    
    config = presets[preset]
    return rag_generate_meal_plan(**config)


print("✅ Meal Plan Generation functions ready!")
print("\nExample usage:")
print("  # Custom meal plan")
print("  plan = rag_generate_meal_plan(days=2, calorie_limit=2000, carb_limit=200, protein_target=100)")
print("\n  # Quick presets")
print("  plan = quick_meal_plan('balanced')")
print("  plan = quick_meal_plan('low_carb')")
print("  plan = quick_meal_plan('high_protein')")


In [ ]:
# Test with custom parameters
result = rag_generate_meal_plan(
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=100,
    dietary_preferences="balanced healthy",
    top_k=30
)

if result:
    print(f"\n✅ Meal plan generated successfully!")
    print(f"📊 Used {result['available_foods_count']} foods from database")


In [ ]:
# Meal Plan Generator from Personal Inventory
def rag_generate_meal_plan_from_inventory(
    inventory: list,
    days: int = 2,
    calorie_limit: int = 2000,
    carb_limit: int = 200,
    protein_target: int = None,
    dietary_preferences: str = None
):
    """
    Generate a meal plan using ONLY foods from your inventory.
    Fetches exact nutritional data from the database.
    
    Args:
        inventory: List of food names you have available
                  Example: ["eggs", "chicken breast", "brown rice", "spinach", ...]
        days: Number of days for meal plan (default: 2)
        calorie_limit: Daily calorie limit
        carb_limit: Daily carb limit in grams
        protein_target: Target protein in grams (optional)
        dietary_preferences: Additional preferences
    """
    print(f"🍽️ Generating {days}-Day Meal Plan from YOUR Inventory")
    print("="*70)
    print(f"📦 Inventory items: {len(inventory)}")
    print(f"📊 Constraints: {calorie_limit} cal/day, {carb_limit}g carbs/day")
    if protein_target:
        print(f"🥩 Protein target: {protein_target}g/day")
    print("="*70)
    
    # Step 1: Find matching foods in database using semantic search
    print(f"\n📚 Step 1: Finding your foods in database...")
    
    matched_foods = []
    food_search_results = {}
    
    for food_item in inventory:
        # Search database for this inventory item
        results = search_snowflake(food_item, top_k=3)
        if not results.empty:
            # Take the best match (highest similarity)
            best_match = results.iloc[0]
            if best_match['SIMILARITY'] > 0.6:  # Good match threshold
                matched_foods.append(best_match['FOOD_NAME'])
                food_search_results[food_item] = best_match['FOOD_NAME']
                print(f"  ✓ '{food_item}' → '{best_match['FOOD_NAME']}' (score: {best_match['SIMILARITY']:.3f})")
            else:
                print(f"  ⚠ '{food_item}' - no good match found")
    
    if len(matched_foods) < 5:
        print(f"\n❌ Error: Only found {len(matched_foods)} foods in database. Need at least 5.")
        print("   Please check your inventory items or add more foods.")
        return None
    
    print(f"\n✅ Matched {len(matched_foods)} foods from your inventory")
    
    # Step 2: Fetch detailed nutrition data for matched foods
    print(f"\n📊 Step 2: Fetching nutritional data from database...")
    
    food_names_escaped = ["'" + name.replace("'", "''") + "'" for name in matched_foods]
    food_names_str = ','.join(food_names_escaped)
    
    nutrition_query = f"""
    SELECT 
        FOOD_NAME,
        CATEGORY,
        COALESCE(CALORIES, 0) as CALORIES,
        COALESCE(PROTEIN, 0) as PROTEIN,
        COALESCE(CARBOHYDRATE, 0) as CARBS,
        COALESCE(TOTAL_FAT, 0) as FAT,
        COALESCE(FIBER, 0) as FIBER,
        COALESCE(SODIUM, 0) as SODIUM
    FROM {source_table}
    WHERE FOOD_NAME IN ({food_names_str})
    """
    
    try:
        nutrition_data = session.sql(nutrition_query).to_pandas()
        print(f"✅ Retrieved nutrition data for {len(nutrition_data)} foods")
        
        # Show sample of nutrition data
        print("\n📋 Sample nutrition data from your inventory:")
        for idx, row in nutrition_data.head(5).iterrows():
            print(f"  • {row['FOOD_NAME'][:40]}: {row['CALORIES']}cal, "
                  f"{row['PROTEIN']}g protein, {row['CARBS']}g carbs")
        
    except Exception as e:
        print(f"❌ Error fetching nutrition data: {e}")
        return None
    
    # Step 3: Build context for LLM with exact nutrition values
    print("\n📝 Step 3: Building meal plan context...")
    
    food_context = []
    for _, row in nutrition_data.iterrows():
        food_context.append(
            f"- {row['FOOD_NAME']} (Category: {row['CATEGORY']}): "
            f"{row['CALORIES']} cal, {row['PROTEIN']}g protein, "
            f"{row['CARBS']}g carbs, {row['FAT']}g fat, {row['FIBER']}g fiber"
        )
    
    context = "\n".join(food_context)
    
    # Step 4: Generate meal plan using Snowflake Cortex
    print("\n🤖 Step 4: Generating personalized meal plan...")
    
    prompt = f"""You are a professional meal planner creating a personalized {days}-day meal plan.

AVAILABLE FOODS (From User's Inventory - with exact nutritional data):
{context}

REQUIREMENTS:
- Create a {days}-day meal plan
- Daily calorie limit: {calorie_limit} calories
- Daily carb limit: {carb_limit} grams
{f'- Daily protein target: {protein_target} grams' if protein_target else ''}
{f'- Dietary preference: {dietary_preferences}' if dietary_preferences else ''}
- Must include: Breakfast, Lunch, Snacks, Dinner for each day
- ONLY use foods from the user's available inventory above
- Use the EXACT nutritional values provided
- Stay within daily limits

OUTPUT FORMAT:
For each day, provide:

**Day [X]:**

**Breakfast:**
- [Food name] ([portion size, e.g., "100g", "1 cup", "2 pieces"])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Lunch:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Snacks:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Dinner:**
- [Food name] ([portion size])
  Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

**Day [X] Totals:**
- Total Calories: [X] / {calorie_limit} cal ({"Within" if "[X]" else "Over"} limit)
- Total Protein: [X]g
- Total Carbs: [X] / {carb_limit}g ({"Within" if "[X]" else "Over"} limit)
- Total Fat: [X]g
- Status: ✓ Within limits / ✗ Over limits

CALCULATION RULES:
1. Use the exact nutritional values per 100g from the database
2. Scale proportionally based on portion size (e.g., 150g = 1.5x the 100g values)
3. Sum all meal components accurately
4. Verify totals are within constraints
5. Distribute calories: breakfast 25%, lunch 30%, snacks 15%, dinner 30%
6. Use realistic, measurable portions
7. Vary foods across days when possible

Generate the complete {days}-day meal plan with precise calculations:"""

    try:
        # Call Snowflake Cortex Complete
        llm_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{prompt.replace("'", "''")}'
        ) as MEAL_PLAN
        """
        
        response = session.sql(llm_query).collect()[0]['MEAL_PLAN']
        
        print("\n" + "="*70)
        print(f"📋 YOUR PERSONALIZED {days}-DAY MEAL PLAN")
        print("="*70)
        print(response)
        print("="*70)
        
        # Show mapping of what was used
        print(f"\n📦 Inventory Mapping:")
        print("-"*70)
        for user_item, db_food in food_search_results.items():
            print(f"  '{user_item}' → '{db_food}'")
        
        return {
            'days': days,
            'calorie_limit': calorie_limit,
            'carb_limit': carb_limit,
            'protein_target': protein_target,
            'user_inventory': inventory,
            'matched_foods': matched_foods,
            'inventory_mapping': food_search_results,
            'available_foods_count': len(nutrition_data),
            'meal_plan': response,
            'nutrition_data': nutrition_data
        }
        
    except Exception as e:
        print(f"❌ Error generating meal plan: {e}")
        return None


print("✅ Inventory-based meal plan generator ready!")
print("\nExample usage:")
print("""
my_inventory = [
    "eggs", "chicken breast", "salmon", 
    "brown rice", "quinoa", "sweet potato",
    "spinach", "broccoli", "carrots",
    "greek yogurt", "almonds", "banana",
    "avocado", "olive oil", "oatmeal"
]

plan = rag_generate_meal_plan_from_inventory(
    inventory=my_inventory,
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=120
)
""")


In [ ]:
# Define YOUR available foods
my_kitchen_inventory = [
    # Proteins
    "eggs",
    "chicken breast", 
    "salmon",
    "greek yogurt",
    "cottage cheese",
    "tofu",
    
    # Carbs
    "brown rice",
    "quinoa",
    "oatmeal",
    "whole wheat bread",
    "sweet potato",
    "pasta",
    
    # Vegetables
    "spinach",
    "broccoli",
    "carrots",
    "bell peppers",
    "tomatoes",
    "cucumber",
    
    # Fruits
    "banana",
    "apple",
    "berries",
    "orange",
    
    # Healthy fats
    "avocado",
    "almonds",
    "olive oil",
    "peanut butter",
    
    # Other
    "milk",
    "cheese",
    "beans"
]

# Generate meal plan
result = rag_generate_meal_plan_from_inventory(
    inventory=my_kitchen_inventory,
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=120,
    dietary_preferences="balanced healthy"
)

if result:
    print(f"\n✅ Success! Meal plan created using {result['available_foods_count']} foods from your inventory")
    
    # Save nutrition data for reference
    print("\n📊 Nutrition reference table:")
    print(result['nutrition_data'][['FOOD_NAME', 'CALORIES', 'PROTEIN', 'CARBS', 'FAT']].head(10))


In [ ]:
# Meal Plan Generator from Inventory WITH RECIPES
def rag_generate_meal_plan_with_recipes(
    inventory: list,
    days: int = 2,
    calorie_limit: int = 2000,
    carb_limit: int = 200,
    protein_target: int = None,
    dietary_preferences: str = None
):
    """
    Generate a meal plan with RECIPES using foods from your inventory.
    Includes cooking instructions, prep time, and nutrition calculations.
    
    Args:
        inventory: List of food names you have available
        days: Number of days for meal plan
        calorie_limit: Daily calorie limit
        carb_limit: Daily carb limit in grams
        protein_target: Target protein in grams (optional)
        dietary_preferences: Additional preferences
    """
    print(f"🍽️ Generating {days}-Day Meal Plan WITH RECIPES")
    print("="*70)
    print(f"📦 Inventory items: {len(inventory)}")
    print(f"📊 Constraints: {calorie_limit} cal/day, {carb_limit}g carbs/day")
    if protein_target:
        print(f"🥩 Protein target: {protein_target}g/day")
    print("="*70)
    
    # Step 1: Find matching foods in database
    print(f"\n📚 Step 1: Finding your foods in database...")
    
    matched_foods = []
    food_search_results = {}
    
    for food_item in inventory:
        results = search_snowflake(food_item, top_k=3)
        if not results.empty:
            best_match = results.iloc[0]
            if best_match['SIMILARITY'] > 0.6:
                matched_foods.append(best_match['FOOD_NAME'])
                food_search_results[food_item] = best_match['FOOD_NAME']
                print(f"  ✓ '{food_item}' → '{best_match['FOOD_NAME']}' (score: {best_match['SIMILARITY']:.3f})")
    
    if len(matched_foods) < 5:
        print(f"\n❌ Error: Only found {len(matched_foods)} foods. Need at least 5.")
        return None
    
    print(f"\n✅ Matched {len(matched_foods)} foods from your inventory")
    
    # Step 2: Fetch nutrition data
    print(f"\n📊 Step 2: Fetching nutritional data...")
    
    food_names_escaped = ["'" + name.replace("'", "''") + "'" for name in matched_foods]
    food_names_str = ','.join(food_names_escaped)
    
    nutrition_query = f"""
    SELECT 
        FOOD_NAME,
        CATEGORY,
        COALESCE(CALORIES, 0) as CALORIES,
        COALESCE(PROTEIN, 0) as PROTEIN,
        COALESCE(CARBOHYDRATE, 0) as CARBS,
        COALESCE(TOTAL_FAT, 0) as FAT,
        COALESCE(FIBER, 0) as FIBER
    FROM {source_table}
    WHERE FOOD_NAME IN ({food_names_str})
    """
    
    try:
        nutrition_data = session.sql(nutrition_query).to_pandas()
        print(f"✅ Retrieved nutrition data for {len(nutrition_data)} foods")
    except Exception as e:
        print(f"❌ Error: {e}")
        return None
    
    # Step 3: Build context
    food_context = []
    for _, row in nutrition_data.iterrows():
        food_context.append(
            f"- {row['FOOD_NAME']} (Category: {row['CATEGORY']}): "
            f"{row['CALORIES']} cal, {row['PROTEIN']}g protein, "
            f"{row['CARBS']}g carbs, {row['FAT']}g fat per 100g"
        )
    
    context = "\n".join(food_context)
    
    # Step 4: Generate meal plan with recipes
    print("\n🤖 Step 3: Generating meal plan with detailed recipes...")
    
    prompt = f"""You are a professional chef and nutritionist creating a detailed {days}-day meal plan with complete recipes.

AVAILABLE INGREDIENTS (From User's Kitchen):
{context}
All the items in the database are per 100g quantitative measure.
REQUIREMENTS:
- Create {days}-day meal plan with COMPLETE RECIPES
- Daily calorie limit: {calorie_limit} calories
- Daily carb limit: {carb_limit} grams
{f'- Daily protein target: {protein_target} grams' if protein_target else ''}
{f'- Dietary preference: {dietary_preferences}' if dietary_preferences else ''}
- Include: Breakfast, Lunch, Snacks, Dinner for each day
- ONLY use ingredients from the available list above
- Provide detailed cooking instructions for each meal

OUTPUT FORMAT:

**Day [X]:**

---

### Breakfast: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])
- [Ingredient 3] ([amount])

**Cooking Instructions:**
1. [Step 1 with specific details]
2. [Step 2 with temperature/time]
3. [Step 3 with cooking technique]
4. [Final step]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Lunch: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])

**Cooking Instructions:**
1. [Detailed step 1]
2. [Detailed step 2]
3. [Detailed step 3]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Snacks: [Recipe Name]
**Prep Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])

**Instructions:**
1. [Simple preparation step]

**Nutrition:**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Dinner: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])
- [Ingredient 3] ([amount])

**Cooking Instructions:**
1. [Detailed step 1]
2. [Detailed step 2]
3. [Detailed step 3]
4. [Final step]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

**Day [X] Totals:**
- Total Calories: [X] / {calorie_limit} cal
- Total Protein: [X]g
- Total Carbs: [X] / {carb_limit}g
- Total Fat: [X]g
- Status: ✓ Within limits

---

IMPORTANT RECIPE GUIDELINES:
1. Give each meal a creative, appetizing name
2. List realistic prep and cook times
3. Provide step-by-step cooking instructions with temperatures and techniques
4. Use exact measurements (cups, grams, tablespoons)
5. Scale nutrition values based on portion sizes
6. Make instructions clear and easy to follow
7. Include cooking methods (sauté, bake, grill, steam, etc.)
8. Suggest seasoning and flavor enhancements using available ingredients
9. Keep recipes simple and practical for home cooking
10. Calculate accurate nutrition totals for each day

Generate the complete {days}-day meal plan with detailed recipes now:"""

    try:
        llm_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{prompt.replace("'", "''")}'
        ) as MEAL_PLAN
        """
        
        response = session.sql(llm_query).collect()[0]['MEAL_PLAN']
        
        print("\n" + "="*70)
        print(f"📋 YOUR {days}-DAY MEAL PLAN WITH RECIPES")
        print("="*70)
        print(response)
        print("="*70)
        
        return {
            'days': days,
            'calorie_limit': calorie_limit,
            'carb_limit': carb_limit,
            'protein_target': protein_target,
            'user_inventory': inventory,
            'matched_foods': matched_foods,
            'inventory_mapping': food_search_results,
            'available_foods_count': len(nutrition_data),
            'meal_plan_with_recipes': response,
            'nutrition_data': nutrition_data
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


print("✅ Recipe-based meal plan generator ready!")
print("\nExample usage:")
print("""
my_inventory = [
    "eggs", "chicken breast", "salmon", "tofu",
    "brown rice", "quinoa", "oatmeal", "pasta",
    "spinach", "broccoli", "tomatoes", "bell peppers",
    "greek yogurt", "milk", "cheese", 
    "almonds", "olive oil", "garlic", "onions"
]

plan = rag_generate_meal_plan_with_recipes(
    inventory=my_inventory,
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=120,
    dietary_preferences="balanced healthy"
)
""")


In [ ]:
# Define your kitchen inventory
my_kitchen = [
    # Proteins
    "eggs", "chicken breast", "Whey Protein (22g per scoup)", "ground turkey",
    "greek yogurt", "cottage cheese", "tofu", "chickpeas",
    
    # Grains & Carbs
    "brown rice", "quinoa", "oatmeal", "whole wheat pasta",
    "sweet potato", "whole wheat bread", "tortillas",
    
    # Vegetables
    "spinach", "broccoli", "carrots", "bell peppers",
    "tomatoes", "cucumber", "zucchini", "kale",
    "onions", "garlic", "mushrooms",
    
    # Fruits
    "banana", "apple", "berries", "orange", "avocado",
    
    # Healthy Fats & Others
    "almonds", "walnuts", "peanut butter", "olive oil",
    "coconut oil", "cheese", "milk", "beans",
    "lentils", "soy sauce", "honey"
]

# Generate 2-day meal plan with complete recipes
result = rag_generate_meal_plan_with_recipes(
    inventory=my_kitchen,
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=120,
    dietary_preferences="balanced healthy meals"
)

if result:
    print(f"\n✅ Complete meal plan with recipes generated!")
    print(f"📊 Using {result['available_foods_count']} ingredients from your kitchen")


In [ ]:
# Meal Plan Generator WITH USER HABITS AND PREFERENCES
def rag_generate_meal_plan_with_habits(
    inventory: list,
    days: int = 2,
    calorie_limit: int = 2000,
    carb_limit: int = 200,
    protein_target: int = None,
    dietary_preferences: str = None,
    daily_habits: dict = None
):
    """
    Generate a meal plan with RECIPES that incorporates user's daily habits.
    
    Args:
        inventory: List of food names you have available
        days: Number of days for meal plan
        calorie_limit: Daily calorie limit
        carb_limit: Daily carb limit in grams
        protein_target: Target protein in grams (optional)
        dietary_preferences: Additional preferences
        daily_habits: Dictionary of fixed habits, e.g.:
                     {
                         "breakfast": "whey protein milkshake",
                         "post_workout": "banana and peanut butter",
                         "snack": "greek yogurt with berries",
                         "avoid": ["dairy", "gluten"]
                     }
    """
    print(f"🍽️ Generating {days}-Day Meal Plan WITH HABITS & RECIPES")
    print("="*70)
    print(f"📦 Inventory items: {len(inventory)}")
    print(f"📊 Constraints: {calorie_limit} cal/day, {carb_limit}g carbs/day")
    if protein_target:
        print(f"🥩 Protein target: {protein_target}g/day")
    if daily_habits:
        print(f"🔄 Daily habits: {len(daily_habits)} configured")
    print("="*70)
    
    # Step 1: Find matching foods in database
    print(f"\n📚 Step 1: Finding your foods in database...")
    
    matched_foods = []
    food_search_results = {}
    
    for food_item in inventory:
        results = search_snowflake(food_item, top_k=3)
        if not results.empty:
            best_match = results.iloc[0]
            if best_match['SIMILARITY'] > 0.6:
                matched_foods.append(best_match['FOOD_NAME'])
                food_search_results[food_item] = best_match['FOOD_NAME']
                print(f"  ✓ '{food_item}' → '{best_match['FOOD_NAME']}' (score: {best_match['SIMILARITY']:.3f})")
    
    if len(matched_foods) < 5:
        print(f"\n❌ Error: Only found {len(matched_foods)} foods. Need at least 5.")
        return None
    
    print(f"\n✅ Matched {len(matched_foods)} foods from your inventory")
    
    # Step 2: Fetch nutrition data
    print(f"\n📊 Step 2: Fetching nutritional data...")
    
    food_names_escaped = ["'" + name.replace("'", "''") + "'" for name in matched_foods]
    food_names_str = ','.join(food_names_escaped)
    
    nutrition_query = f"""
    SELECT 
        FOOD_NAME,
        CATEGORY,
        COALESCE(CALORIES, 0) as CALORIES,
        COALESCE(PROTEIN, 0) as PROTEIN,
        COALESCE(CARBOHYDRATE, 0) as CARBS,
        COALESCE(TOTAL_FAT, 0) as FAT,
        COALESCE(FIBER, 0) as FIBER
    FROM {source_table}
    WHERE FOOD_NAME IN ({food_names_str})
    """
    
    try:
        nutrition_data = session.sql(nutrition_query).to_pandas()
        print(f"✅ Retrieved nutrition data for {len(nutrition_data)} foods")
    except Exception as e:
        print(f"❌ Error: {e}")
        return None
    
    # Step 3: Build context
    food_context = []
    for _, row in nutrition_data.iterrows():
        food_context.append(
            f"- {row['FOOD_NAME']} (Category: {row['CATEGORY']}): "
            f"{row['CALORIES']} cal, {row['PROTEIN']}g protein, "
            f"{row['CARBS']}g carbs, {row['FAT']}g fat per 100g"
        )
    
    context = "\n".join(food_context)
    
    # Step 4: Build habits section for prompt
    habits_section = ""
    if daily_habits:
        habits_section = "\n\nUSER'S DAILY HABITS (MUST INCLUDE):\n"
        for habit_key, habit_value in daily_habits.items():
            if habit_key == "avoid":
                habits_section += f"- Foods to AVOID: {', '.join(habit_value) if isinstance(habit_value, list) else habit_value}\n"
            elif habit_key == "breakfast":
                habits_section += f"- EVERY morning breakfast: {habit_value} (MANDATORY - include this every day)\n"
            elif habit_key == "post_workout":
                habits_section += f"- Post-workout habit: {habit_value}\n"
            elif habit_key == "snack":
                habits_section += f"- Preferred snack: {habit_value}\n"
            else:
                habits_section += f"- {habit_key.replace('_', ' ').title()}: {habit_value}\n"
    
    # Step 5: Generate meal plan with habits
    print("\n🤖 Step 3: Generating personalized meal plan with habits...")
    
    prompt = f"""You are a professional chef and nutritionist creating a {days}-day meal plan with complete recipes.

AVAILABLE INGREDIENTS (From User's Kitchen):
{context}
{habits_section}

REQUIREMENTS:
- Create {days}-day meal plan with COMPLETE RECIPES
- Daily calorie limit: {calorie_limit} calories
- Daily carb limit: {carb_limit} grams
{f'- Daily protein target: {protein_target} grams' if protein_target else ''}
{f'- Dietary preference: {dietary_preferences}' if dietary_preferences else ''}
- Include: Breakfast, Lunch, Snacks, Dinner for each day
- ONLY use ingredients from the available list above
- **STRICTLY FOLLOW user's daily habits (marked MANDATORY must be included every single day)**

OUTPUT FORMAT:

**Day [X]:**

---

### Breakfast: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes
{"**[⭐ DAILY HABIT]**" if daily_habits and "breakfast" in daily_habits else ""}

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])

**Cooking Instructions:**
1. [Detailed step with technique]
2. [Detailed step with time/temp]
3. [Final step]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Lunch: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])

**Cooking Instructions:**
1. [Detailed step]
2. [Detailed step]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Snacks: [Recipe Name]
**Prep Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])

**Instructions:**
1. [Simple preparation]

**Nutrition:**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

### Dinner: [Recipe Name]
**Prep Time:** [X] minutes | **Cook Time:** [X] minutes

**Ingredients:**
- [Ingredient 1] ([amount])
- [Ingredient 2] ([amount])

**Cooking Instructions:**
1. [Detailed step]
2. [Detailed step]

**Nutrition (per serving):**
Calories: [X] cal | Protein: [X]g | Carbs: [X]g | Fat: [X]g

---

**Day [X] Totals:**
- Total Calories: [X] / {calorie_limit} cal
- Total Protein: [X]g
- Total Carbs: [X] / {carb_limit}g
- Total Fat: [X]g
- Status: ✓ Within limits

---

CRITICAL RULES:
1. If user has a MANDATORY breakfast habit (like "whey protein milkshake"), include it EVERY single day as breakfast
2. Provide complete recipe for habit items (e.g., for whey shake: list whey powder, milk, banana amounts)
3. Calculate nutrition for habit items and subtract from daily totals when planning other meals
4. Give creative names to each meal
5. List realistic prep/cook times
6. Provide step-by-step instructions with temperatures and techniques
7. Use exact measurements (cups, grams, tablespoons)
8. Scale nutrition based on portion sizes
9. Account for user habits in calorie/macro distribution
10. Make recipes simple and practical

Generate the complete {days}-day meal plan with detailed recipes now:"""

    try:
        llm_query = f"""
        SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            '{prompt.replace("'", "''")}'
        ) as MEAL_PLAN
        """
        
        response = session.sql(llm_query).collect()[0]['MEAL_PLAN']
        
        print("\n" + "="*70)
        print(f"📋 YOUR PERSONALIZED {days}-DAY MEAL PLAN")
        print("="*70)
        print(response)
        print("="*70)
        
        # Display habits summary
        if daily_habits:
            print(f"\n🔄 Daily Habits Applied:")
            print("-"*70)
            for habit_key, habit_value in daily_habits.items():
                print(f"  • {habit_key.replace('_', ' ').title()}: {habit_value}")
        
        return {
            'days': days,
            'calorie_limit': calorie_limit,
            'carb_limit': carb_limit,
            'protein_target': protein_target,
            'daily_habits': daily_habits,
            'user_inventory': inventory,
            'matched_foods': matched_foods,
            'inventory_mapping': food_search_results,
            'available_foods_count': len(nutrition_data),
            'meal_plan_with_recipes': response,
            'nutrition_data': nutrition_data
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


print("✅ Habit-aware meal plan generator ready!")
print("\nExample usage:")
print("""
# Define your habits
my_habits = {
    "breakfast": "whey protein milkshake with banana",
    "snack": "greek yogurt with almonds",
    "avoid": ["dairy", "gluten"]  # optional
}

# Your inventory
my_inventory = [
    "whey protein powder", "milk", "banana", "eggs",
    "chicken breast", "salmon", "brown rice", "quinoa",
    "spinach", "broccoli", "greek yogurt", "almonds"
]

plan = rag_generate_meal_plan_with_habits(
    inventory=my_inventory,
    days=2,
    calorie_limit=2000,
    carb_limit=200,
    protein_target=120,
    dietary_preferences="balanced healthy",
    daily_habits=my_habits
)
""")


In [ ]:
# Define your daily habits
my_daily_habits = {
    "breakfast": "whey protein milkshake with banana and milk",  # This will be EVERY morning
    "snack": "greek yogurt with berries",
    "post_workout": "banana with peanut butter"
}

# Your kitchen inventory
my_inventory = [
    # For your daily shake
    "whey protein powder", "milk", "banana", "berries",
    
    # Proteins
    "eggs", "chicken breast", "salmon", "ground turkey",
    "greek yogurt", "cottage cheese", "tofu",
    
    # Carbs
    "brown rice", "quinoa", "oatmeal", "sweet potato",
    "whole wheat bread", "pasta",
    
    # Vegetables
    "spinach", "broccoli", "carrots", "bell peppers",
    "tomatoes", "zucchini", "onions", "garlic",
    
    # Fats & Others
    "peanut butter", "almonds", "olive oil", "avocado",
    "cheese", "butter", "honey"
]

# Generate meal plan with your habits
result = rag_generate_meal_plan_with_habits(
    inventory=my_inventory,
    days=2,
    calorie_limit=2200,  # Higher since you have protein shake
    carb_limit=220,
    protein_target=150,  # Higher protein target for fitness
    dietary_preferences="high protein balanced meals",
    daily_habits=my_daily_habits
)

if result:
    print(f"\n✅ Personalized meal plan created!")
    print(f"📊 Using {result['available_foods_count']} ingredients")
    print(f"🔄 Daily habits incorporated: {len(result['daily_habits'])}")


In [ ]:
# Complex habits configuration
advanced_habits = {
    "breakfast": "whey protein milkshake with banana and oats (30g protein)",
    "pre_workout": "black coffee with banana",
    "post_workout": "protein shake with berries",
    "evening_snack": "cottage cheese with almonds",
    "avoid": ["dairy in dinner", "fried foods", "processed sugar"],
    "timing": "early dinner by 7 PM"
}

advanced_plan = rag_generate_meal_plan_with_habits(
    inventory=my_inventory,
    days=3,
    calorie_limit=2400,
    carb_limit=250,
    protein_target=160,
    dietary_preferences="bodybuilding meal prep",
    daily_habits=advanced_habits
)


In [ ]:
print("\n" + "="*70)
print("📊 ADVANCED QUANTITATIVE ANALYSIS")
print("="*70)

import numpy as np
from scipy import stats
import time

# Initialize results dictionary
quantitative_metrics = {}

In [ ]:
print("\n1️⃣ EMBEDDING DISTRIBUTION ANALYSIS")
print("-"*50)

# Get sample embeddings for analysis (100 from each model)
snowflake_sample = session.sql(f"""
    SELECT EMBEDDING
    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS
    LIMIT 100
""").to_pandas()

openai_sample = session.sql(f"""
    SELECT EMBEDDING
    FROM {embeddings_schema}.OPENAI_EMBEDDINGS
    WHERE EMBEDDING IS NOT NULL
    LIMIT 100
""").to_pandas()

if len(snowflake_sample) > 0:
    # Convert embeddings to numpy arrays
    snow_embeddings = np.array([eval(str(e)) if isinstance(e, str) else e for e in snowflake_sample['EMBEDDING'].values])
    
    # Calculate statistics for Snowflake
    snow_means = np.mean(snow_embeddings, axis=0)
    snow_stds = np.std(snow_embeddings, axis=0)
    
    print("📍 Snowflake Arctic Embedding Statistics:")
    print(f"   • Mean of means: {np.mean(snow_means):.6f}")
    print(f"   • Std of means: {np.std(snow_means):.6f}")
    print(f"   • Mean of stds: {np.mean(snow_stds):.6f}")
    print(f"   • Embedding norm (avg): {np.mean([np.linalg.norm(e) for e in snow_embeddings]):.4f}")
    
    quantitative_metrics['snowflake_distribution'] = {
        'mean_of_means': float(np.mean(snow_means)),
        'std_of_means': float(np.std(snow_means)),
        'mean_of_stds': float(np.mean(snow_stds)),
        'avg_norm': float(np.mean([np.linalg.norm(e) for e in snow_embeddings]))
    }

if len(openai_sample) > 0:
    # Similar for OpenAI
    openai_embeddings = np.array([eval(str(e)) if isinstance(e, str) else e for e in openai_sample['EMBEDDING'].values])
    
    openai_means = np.mean(openai_embeddings, axis=0)
    openai_stds = np.std(openai_embeddings, axis=0)
    
    print("\n📍 OpenAI Embedding Statistics:")
    print(f"   • Mean of means: {np.mean(openai_means):.6f}")
    print(f"   • Std of means: {np.std(openai_means):.6f}")
    print(f"   • Mean of stds: {np.mean(openai_stds):.6f}")
    print(f"   • Embedding norm (avg): {np.mean([np.linalg.norm(e) for e in openai_embeddings]):.4f}")
    
    quantitative_metrics['openai_distribution'] = {
        'mean_of_means': float(np.mean(openai_means)),
        'std_of_means': float(np.std(openai_means)),
        'mean_of_stds': float(np.mean(openai_stds)),
        'avg_norm': float(np.mean([np.linalg.norm(e) for e in openai_embeddings]))
    }

In [ ]:
print("\n2️⃣ RETRIEVAL QUALITY METRICS")
print("-"*50)

# Test queries for retrieval metrics
test_queries_retrieval = [
    "high protein vegetarian",
    "low calorie breakfast", 
    "healthy snacks",
    "mediterranean diet",
    "gluten free options"
]

precision_at_k = {'snowflake': [], 'openai': []}
recall_scores = {'snowflake': [], 'openai': []}

for query in test_queries_retrieval:
    # Get top 10 results from each model
    snow_results = search_snowflake(query, top_k=10)
    openai_results = search_openai(query, top_k=10)
    
    if not snow_results.empty and not openai_results.empty:
        # Calculate overlap (as proxy for precision)
        snow_foods = set(snow_results['FOOD_NAME'].tolist()[:5])
        openai_foods = set(openai_results['FOOD_NAME'].tolist()[:5])
        
        # Precision@5 (overlap as relevance proxy)
        overlap = len(snow_foods.intersection(openai_foods))
        precision_at_k['snowflake'].append(overlap / 5)
        precision_at_k['openai'].append(overlap / 5)
        
        # Check if results contain query terms (simple relevance check)
        query_terms = query.lower().split()
        
        snow_relevant = sum(1 for food in snow_foods if any(term in food.lower() for term in query_terms))
        openai_relevant = sum(1 for food in openai_foods if any(term in food.lower() for term in query_terms))
        
        recall_scores['snowflake'].append(snow_relevant / 5)
        recall_scores['openai'].append(openai_relevant / 5)

if precision_at_k['snowflake']:
    print("📍 Precision@5 (Average):")
    print(f"   • Snowflake: {np.mean(precision_at_k['snowflake']):.3f}")
    print(f"   • OpenAI: {np.mean(precision_at_k['openai']):.3f}")
    
    print("\n📍 Relevance Score (Query term matching):")
    print(f"   • Snowflake: {np.mean(recall_scores['snowflake']):.3f}")
    print(f"   • OpenAI: {np.mean(recall_scores['openai']):.3f}")
    
    quantitative_metrics['retrieval_quality'] = {
        'snowflake_precision_at_5': float(np.mean(precision_at_k['snowflake'])),
        'openai_precision_at_5': float(np.mean(precision_at_k['openai'])),
        'snowflake_relevance': float(np.mean(recall_scores['snowflake'])),
        'openai_relevance': float(np.mean(recall_scores['openai']))
    }

In [ ]:
print("\n3️⃣ SEMANTIC COHERENCE ANALYSIS")
print("-"*50)

# Get embeddings for specific food categories
categories_to_test = ['Vegetables', 'Fruits', 'Dairy', 'Grains']
intra_category_similarity = {'snowflake': [], 'openai': []}
inter_category_similarity = {'snowflake': [], 'openai': []}

for category in categories_to_test[:2]:  # Test first 2 categories for speed
    # Get embeddings for this category from Snowflake
    snow_category = session.sql(f"""
        SELECT EMBEDDING
        FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS
        WHERE CATEGORY = '{category}'
        LIMIT 20
    """).to_pandas()
    
    if len(snow_category) >= 10:
        embeddings = np.array([eval(str(e)) if isinstance(e, str) else e for e in snow_category['EMBEDDING'].values[:10]])
        
        # Calculate intra-category similarity (should be high for good embeddings)
        similarities = []
        for i in range(len(embeddings)):
            for j in range(i+1, len(embeddings)):
                cos_sim = np.dot(embeddings[i], embeddings[j]) / (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
                similarities.append(cos_sim)
        
        if similarities:
            intra_category_similarity['snowflake'].append(np.mean(similarities))
    
    # Same for OpenAI
    openai_category = session.sql(f"""
        SELECT EMBEDDING
        FROM {embeddings_schema}.OPENAI_EMBEDDINGS
        WHERE CATEGORY = '{category}'
        LIMIT 20
    """).to_pandas()
    
    if len(openai_category) >= 10:
        embeddings = np.array([eval(str(e)) if isinstance(e, str) else e for e in openai_category['EMBEDDING'].values[:10]])
        
        similarities = []
        for i in range(len(embeddings)):
            for j in range(i+1, len(embeddings)):
                cos_sim = np.dot(embeddings[i], embeddings[j]) / (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j]))
                similarities.append(cos_sim)
        
        if similarities:
            intra_category_similarity['openai'].append(np.mean(similarities))

if intra_category_similarity['snowflake']:
    print("📍 Intra-Category Similarity (Higher is better):")
    print(f"   • Snowflake: {np.mean(intra_category_similarity['snowflake']):.4f}")
    if intra_category_similarity['openai']:
        print(f"   • OpenAI: {np.mean(intra_category_similarity['openai']):.4f}")
    
    quantitative_metrics['semantic_coherence'] = {
        'snowflake_intra_similarity': float(np.mean(intra_category_similarity['snowflake'])),
        'openai_intra_similarity': float(np.mean(intra_category_similarity['openai'])) if intra_category_similarity['openai'] else 0
    }

In [ ]:
print("\n4️⃣ RESPONSE TIME & THROUGHPUT ANALYSIS")
print("-"*50)

# Measure search response times
search_times_snow = []
search_times_openai = []
test_search_queries = ["protein", "low calorie", "vegetarian", "breakfast", "snack"]

for q in test_search_queries:
    # Snowflake search time
    start = time.time()
    _ = search_snowflake(q, top_k=10)
    search_times_snow.append((time.time() - start) * 1000)
    
    # OpenAI search time
    if client:
        start = time.time()
        _ = search_openai(q, top_k=10)
        search_times_openai.append((time.time() - start) * 1000)

print("📍 Search Response Times (ms):")
print(f"   • Snowflake:")
print(f"     - Mean: {np.mean(search_times_snow):.2f}ms")
print(f"     - Median: {np.median(search_times_snow):.2f}ms")
print(f"     - P95: {np.percentile(search_times_snow, 95):.2f}ms")

if search_times_openai:
    print(f"   • OpenAI:")
    print(f"     - Mean: {np.mean(search_times_openai):.2f}ms")
    print(f"     - Median: {np.median(search_times_openai):.2f}ms")
    print(f"     - P95: {np.percentile(search_times_openai, 95):.2f}ms")

# Calculate throughput
snow_throughput = 1000 / np.mean(search_times_snow) if search_times_snow else 0  # queries per second
openai_throughput = 1000 / np.mean(search_times_openai) if search_times_openai else 0

print(f"\n📍 Theoretical Throughput:")
print(f"   • Snowflake: {snow_throughput:.1f} queries/second")
if search_times_openai:
    print(f"   • OpenAI: {openai_throughput:.1f} queries/second")

quantitative_metrics['performance'] = {
    'snowflake_mean_search_ms': float(np.mean(search_times_snow)),
    'snowflake_p95_search_ms': float(np.percentile(search_times_snow, 95)),
    'snowflake_throughput_qps': float(snow_throughput),
    'openai_mean_search_ms': float(np.mean(search_times_openai)) if search_times_openai else 0,
    'openai_p95_search_ms': float(np.percentile(search_times_openai, 95)) if search_times_openai else 0,
    'openai_throughput_qps': float(openai_throughput)
}

In [ ]:
print("\n6️⃣ DIVERSITY & COVERAGE METRICS")
print("-"*50)

# Analyze unique tokens/concepts covered
unique_categories_snow = session.sql(f"""
    SELECT COUNT(DISTINCT CATEGORY) as cnt 
    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS 
    WHERE CATEGORY IS NOT NULL
""").collect()[0]['CNT']

unique_categories_openai = session.sql(f"""
    SELECT COUNT(DISTINCT CATEGORY) as cnt 
    FROM {embeddings_schema}.OPENAI_EMBEDDINGS 
    WHERE CATEGORY IS NOT NULL
""").collect()[0]['CNT']

# Get diversity in search results
diversity_queries = ["food", "healthy", "meal"]
diversity_scores = {'snowflake': [], 'openai': []}

for query in diversity_queries:
    snow_results = search_snowflake(query, top_k=20)
    if not snow_results.empty:
        unique_categories_in_results = len(snow_results['CATEGORY'].dropna().unique())
        diversity_scores['snowflake'].append(unique_categories_in_results / 20)
    
    openai_results = search_openai(query, top_k=20)
    if not openai_results.empty:
        unique_categories_in_results = len(openai_results['CATEGORY'].dropna().unique())
        diversity_scores['openai'].append(unique_categories_in_results / 20)

print("📍 Coverage Metrics:")
print(f"   • Unique categories indexed:")
print(f"     - Snowflake: {unique_categories_snow}")
print(f"     - OpenAI: {unique_categories_openai}")

if diversity_scores['snowflake']:
    print("\n📍 Result Diversity (avg unique categories in top-20):")
    print(f"   • Snowflake: {np.mean(diversity_scores['snowflake']):.3f}")
    if diversity_scores['openai']:
        print(f"   • OpenAI: {np.mean(diversity_scores['openai']):.3f}")

quantitative_metrics['diversity'] = {
    'snowflake_categories_covered': int(unique_categories_snow),
    'openai_categories_covered': int(unique_categories_openai),
    'snowflake_result_diversity': float(np.mean(diversity_scores['snowflake'])) if diversity_scores['snowflake'] else 0,
    'openai_result_diversity': float(np.mean(diversity_scores['openai'])) if diversity_scores['openai'] else 0
}

In [ ]:
print("\n7️⃣ STATISTICAL SIGNIFICANCE TESTING")
print("-"*50)

# Compare search times for statistical significance
if len(search_times_snow) > 1 and len(search_times_openai) > 1:
    t_stat, p_value = stats.ttest_ind(search_times_snow, search_times_openai)
    
    print("📍 Response Time Comparison (t-test):")
    print(f"   • t-statistic: {t_stat:.4f}")
    print(f"   • p-value: {p_value:.4f}")
    print(f"   • Significant difference: {'Yes' if p_value < 0.05 else 'No'} (α=0.05)")
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt((np.var(search_times_snow) + np.var(search_times_openai)) / 2)
    cohens_d = (np.mean(search_times_snow) - np.mean(search_times_openai)) / pooled_std
    
    print(f"\n📍 Effect Size (Cohen's d): {abs(cohens_d):.3f}")
    print(f"   • Interpretation: ", end="")
    if abs(cohens_d) < 0.2:
        print("Negligible")
    elif abs(cohens_d) < 0.5:
        print("Small")
    elif abs(cohens_d) < 0.8:
        print("Medium")
    else:
        print("Large")
    
    quantitative_metrics['statistical_tests'] = {
        't_statistic': float(t_stat),
        'p_value': float(p_value),
        'cohens_d': float(cohens_d),
        'significant_difference': p_value < 0.05
    }


LLM-Based Quantitative Evaluation

In [ ]:
print("\n" + "="*70)
print("🤖 LLM-BASED QUANTITATIVE EVALUATION")
print("="*70)

import json
import time
import numpy as np
from typing import Dict, List, Tuple

# Initialize LLM metrics storage
llm_metrics = {
    'rag_quality': {},
    'response_coherence': {},
    'factual_accuracy': {},
    'context_relevance': {},
    'generation_metrics': {}
}

# Define evaluation queries for different meal planning scenarios
evaluation_queries = [
    {
        'query': 'Create a high-protein vegetarian breakfast menu',
        'expected_keywords': ['protein', 'vegetarian', 'breakfast', 'eggs', 'dairy', 'legumes'],
        'nutrition_focus': 'protein'
    },
    {
        'query': 'Suggest low-calorie snacks under 200 calories',
        'expected_keywords': ['calories', 'snack', 'low', 'light', 'healthy'],
        'nutrition_focus': 'calories'
    },
    {
        'query': 'Design a Mediterranean diet lunch plan',
        'expected_keywords': ['mediterranean', 'olive', 'fish', 'vegetables', 'whole grains'],
        'nutrition_focus': 'balanced'
    },
    {
        'query': 'Recommend foods for muscle recovery after workout',
        'expected_keywords': ['protein', 'recovery', 'muscle', 'carbs', 'amino'],
        'nutrition_focus': 'recovery'
    },
    {
        'query': 'Find gluten-free dinner options',
        'expected_keywords': ['gluten-free', 'rice', 'quinoa', 'vegetables', 'meat'],
        'nutrition_focus': 'dietary_restriction'
    }
]

print(f"📋 Prepared {len(evaluation_queries)} evaluation queries")
print("   Categories: Protein focus, Calorie control, Diet styles, Recovery, Restrictions")


In [ ]:
print("\n1️⃣ RAG QUALITY EVALUATION - SNOWFLAKE ARCTIC")
print("-"*50)

snowflake_rag_scores = {
    'relevance_scores': [],
    'response_times': [],
    'context_quality': [],
    'keyword_coverage': [],
    'factual_accuracy': []
}

for eval_item in evaluation_queries[:3]:  # Test first 3 queries for demo
    query = eval_item['query']
    expected_keywords = eval_item['expected_keywords']
    
    print(f"\n📝 Query: '{query}'")
    
    start_time = time.time()
    
    # Step 1: Retrieve relevant foods using Snowflake embeddings
    try:
        results = session.sql(f"""
            WITH query_embed AS (
                SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                    'snowflake-arctic-embed-m', 
                    '{query.replace("'", "''")}' 
                ) as QUERY_EMBEDDING
            )
            SELECT 
                s.FDC_ID,
                s.FOOD_NAME,
                s.CATEGORY,
                m.CALORIES,
                m.PROTEIN,
                m.CARBOHYDRATE,
                m.TOTAL_FAT,
                VECTOR_COSINE_SIMILARITY(
                    s.EMBEDDING, 
                    q.QUERY_EMBEDDING
                ) as SIMILARITY
            FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s
            JOIN {source_table} m ON s.FDC_ID = m.FDC_ID,
                 query_embed q
            ORDER BY SIMILARITY DESC
            LIMIT 5
        """).to_pandas()
        
        retrieval_time = (time.time() - start_time) * 1000
        
        # Step 2: Prepare context for LLM
        context_items = []
        for _, row in results.iterrows():
            context_items.append(
                f"- {row['FOOD_NAME']} ({row['CATEGORY']}): "
                f"Calories: {row['CALORIES']:.0f}, "
                f"Protein: {row['PROTEIN']:.1f}g, "
                f"Carbs: {row['CARBOHYDRATE']:.1f}g, "
                f"Fat: {row['TOTAL_FAT']:.1f}g"
            )
        context = "\n".join(context_items)
        
        # Step 3: Generate response with LLM
        llm_prompt = f"""
        Based on the following foods retrieved from our database:
        {context}
        
        User Query: {query}
        
        Provide a detailed meal recommendation that addresses the query.
        Include specific foods from the list and explain why they fit the requirements.
        """
        
        llm_start = time.time()
        llm_response = session.sql(f"""
            SELECT SNOWFLAKE.CORTEX.COMPLETE(
                'mistral-7b',
                '{llm_prompt.replace("'", "''")}'
            ) as RESPONSE
        """).collect()[0]['RESPONSE']
        llm_time = (time.time() - llm_start) * 1000
        
        # Step 4: Evaluate response quality
        # Check keyword coverage
        response_lower = llm_response.lower()
        keywords_found = sum(1 for keyword in expected_keywords if keyword in response_lower)
        keyword_coverage = keywords_found / len(expected_keywords)
        
        # Check if retrieved foods are mentioned in response
        foods_mentioned = sum(1 for _, row in results.iterrows() 
                             if row['FOOD_NAME'].lower() in response_lower)
        context_usage = foods_mentioned / len(results) if len(results) > 0 else 0
        
        # Calculate average similarity score (relevance)
        avg_similarity = results['SIMILARITY'].mean()
        
        # Store metrics
        snowflake_rag_scores['relevance_scores'].append(avg_similarity)
        snowflake_rag_scores['response_times'].append(retrieval_time + llm_time)
        snowflake_rag_scores['context_quality'].append(context_usage)
        snowflake_rag_scores['keyword_coverage'].append(keyword_coverage)
        
        print(f"   ✅ Relevance: {avg_similarity:.4f}")
        print(f"   ✅ Keyword coverage: {keyword_coverage:.2%}")
        print(f"   ✅ Context usage: {context_usage:.2%}")
        print(f"   ✅ Total time: {retrieval_time + llm_time:.2f}ms")
        
    except Exception as e:
        print(f"   ⚠️ Error: {str(e)}")

# Calculate aggregate metrics
if snowflake_rag_scores['relevance_scores']:
    llm_metrics['snowflake_rag'] = {
        'avg_relevance': float(np.mean(snowflake_rag_scores['relevance_scores'])),
        'avg_response_time_ms': float(np.mean(snowflake_rag_scores['response_times'])),
        'avg_context_usage': float(np.mean(snowflake_rag_scores['context_quality'])),
        'avg_keyword_coverage': float(np.mean(snowflake_rag_scores['keyword_coverage']))
    }
    
    print(f"\n📊 Snowflake + LLM Aggregate Scores:")
    print(f"   • Average Relevance: {llm_metrics['snowflake_rag']['avg_relevance']:.4f}")
    print(f"   • Keyword Coverage: {llm_metrics['snowflake_rag']['avg_keyword_coverage']:.2%}")
    print(f"   • Context Usage: {llm_metrics['snowflake_rag']['avg_context_usage']:.2%}")
    print(f"   • Avg Response Time: {llm_metrics['snowflake_rag']['avg_response_time_ms']:.2f}ms")


In [ ]:
print("\n2️⃣ RAG QUALITY EVALUATION - OPENAI")
print("-"*50)

openai_rag_scores = {
    'relevance_scores': [],
    'response_times': [],
    'context_quality': [],
    'keyword_coverage': [],
    'factual_accuracy': []
}

if client is not None:
    for eval_item in evaluation_queries[:3]:  # Test first 3 queries
        query = eval_item['query']
        expected_keywords = eval_item['expected_keywords']
        
        print(f"\n📝 Query: '{query}'")
        
        start_time = time.time()
        
        # Step 1: Get query embedding from OpenAI
        try:
            response = client.embeddings.create(
                model="text-embedding-3-small",
                input=query
            )
            query_embedding = response.data[0].embedding
            embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
            
            # Retrieve relevant foods
            results = session.sql(f"""
                SELECT 
                    o.FDC_ID,
                    o.FOOD_NAME,
                    o.CATEGORY,
                    m.CALORIES,
                    m.PROTEIN,
                    m.CARBOHYDRATE,
                    m.TOTAL_FAT,
                    VECTOR_COSINE_SIMILARITY(
                        o.EMBEDDING,
                        {embedding_str}::VECTOR(FLOAT, 1536)
                    ) as SIMILARITY
                FROM {embeddings_schema}.OPENAI_EMBEDDINGS o
                JOIN {source_table} m ON o.FDC_ID = m.FDC_ID
                ORDER BY SIMILARITY DESC
                LIMIT 5
            """).to_pandas()
            
            retrieval_time = (time.time() - start_time) * 1000
            
            # Step 2: Prepare context for LLM
            context_items = []
            for _, row in results.iterrows():
                context_items.append(
                    f"- {row['FOOD_NAME']} ({row['CATEGORY']}): "
                    f"Calories: {row['CALORIES']:.0f}, "
                    f"Protein: {row['PROTEIN']:.1f}g, "
                    f"Carbs: {row['CARBOHYDRATE']:.1f}g, "
                    f"Fat: {row['TOTAL_FAT']:.1f}g"
                )
            context = "\n".join(context_items)
            
            # Step 3: Generate response with LLM
            llm_prompt = f"""
            Based on the following foods retrieved from our database:
            {context}
            
            User Query: {query}
            
            Provide a detailed meal recommendation that addresses the query.
            Include specific foods from the list and explain why they fit the requirements.
            """
            
            llm_start = time.time()
            llm_response = session.sql(f"""
                SELECT SNOWFLAKE.CORTEX.COMPLETE(
                    'mistral-7b',
                    '{llm_prompt.replace("'", "''")}'
                ) as RESPONSE
            """).collect()[0]['RESPONSE']
            llm_time = (time.time() - llm_start) * 1000
            
            # Step 4: Evaluate response quality
            response_lower = llm_response.lower()
            keywords_found = sum(1 for keyword in expected_keywords if keyword in response_lower)
            keyword_coverage = keywords_found / len(expected_keywords)
            
            foods_mentioned = sum(1 for _, row in results.iterrows() 
                                 if row['FOOD_NAME'].lower() in response_lower)
            context_usage = foods_mentioned / len(results) if len(results) > 0 else 0
            
            avg_similarity = results['SIMILARITY'].mean()
            
            # Store metrics
            openai_rag_scores['relevance_scores'].append(avg_similarity)
            openai_rag_scores['response_times'].append(retrieval_time + llm_time)
            openai_rag_scores['context_quality'].append(context_usage)
            openai_rag_scores['keyword_coverage'].append(keyword_coverage)
            
            print(f"   ✅ Relevance: {avg_similarity:.4f}")
            print(f"   ✅ Keyword coverage: {keyword_coverage:.2%}")
            print(f"   ✅ Context usage: {context_usage:.2%}")
            print(f"   ✅ Total time: {retrieval_time + llm_time:.2f}ms")
            
        except Exception as e:
            print(f"   ⚠️ Error: {str(e)}")
    
    # Calculate aggregate metrics
    if openai_rag_scores['relevance_scores']:
        llm_metrics['openai_rag'] = {
            'avg_relevance': float(np.mean(openai_rag_scores['relevance_scores'])),
            'avg_response_time_ms': float(np.mean(openai_rag_scores['response_times'])),
            'avg_context_usage': float(np.mean(openai_rag_scores['context_quality'])),
            'avg_keyword_coverage': float(np.mean(openai_rag_scores['keyword_coverage']))
        }
        
        print(f"\n📊 OpenAI + LLM Aggregate Scores:")
        print(f"   • Average Relevance: {llm_metrics['openai_rag']['avg_relevance']:.4f}")
        print(f"   • Keyword Coverage: {llm_metrics['openai_rag']['avg_keyword_coverage']:.2%}")
        print(f"   • Context Usage: {llm_metrics['openai_rag']['avg_context_usage']:.2%}")
        print(f"   • Avg Response Time: {llm_metrics['openai_rag']['avg_response_time_ms']:.2f}ms")


In [ ]:
print("\n3️⃣ COMPARATIVE LLM RESPONSE QUALITY")
print("-"*50)

# Test specific scenarios for quality comparison
quality_test_queries = [
    "What should I eat for a high-protein breakfast?",
    "Suggest healthy snacks for weight loss",
    "Create a balanced meal plan for diabetes management"
]

comparative_scores = {
    'snowflake': {'coherence': [], 'completeness': [], 'accuracy': []},
    'openai': {'coherence': [], 'completeness': [], 'accuracy': []}
}

for test_query in quality_test_queries[:2]:  # Test first 2 for demo
    print(f"\n📝 Testing: '{test_query}'")
    
    # Get responses from both pipelines
    for model_name in ['snowflake', 'openai']:
        if model_name == 'openai' and client is None:
            continue
            
        print(f"\n   {model_name.upper()} Pipeline:")
        
        try:
            # Retrieve context
            if model_name == 'snowflake':
                retrieve_query = f"""
                    WITH query_embed AS (
                        SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                            'snowflake-arctic-embed-m', 
                            '{test_query.replace("'", "''")}' 
                        ) as QUERY_EMBEDDING
                    )
                    SELECT 
                        s.FOOD_NAME,
                        s.CATEGORY,
                        VECTOR_COSINE_SIMILARITY(s.EMBEDDING, q.QUERY_EMBEDDING) as SIMILARITY
                    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s, query_embed q
                    ORDER BY SIMILARITY DESC
                    LIMIT 5
                """
            else:  # OpenAI
                response = client.embeddings.create(
                    model="text-embedding-3-small",
                    input=test_query
                )
                query_embedding = response.data[0].embedding
                embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
                
                retrieve_query = f"""
                    SELECT 
                        FOOD_NAME,
                        CATEGORY,
                        VECTOR_COSINE_SIMILARITY(
                            EMBEDDING,
                            {embedding_str}::VECTOR(FLOAT, 1536)
                        ) as SIMILARITY
                    FROM {embeddings_schema}.OPENAI_EMBEDDINGS
                    ORDER BY SIMILARITY DESC
                    LIMIT 5
                """
            
            results = session.sql(retrieve_query).to_pandas()
            
            # Create context
            context = "\n".join([f"- {row['FOOD_NAME']}" for _, row in results.iterrows()])
            
            # Generate LLM response
            llm_response = session.sql(f"""
                SELECT SNOWFLAKE.CORTEX.COMPLETE(
                    'mistral-7b',
                    'Based on these foods: {context.replace("'", "''")}
                    
                    Query: {test_query.replace("'", "''")}
                    
                    Provide a helpful meal recommendation.'
                ) as RESPONSE
            """).collect()[0]['RESPONSE']
            
            # Evaluate response with another LLM call
            eval_prompt = f"""
            Evaluate the following response on a scale of 1-10 for:
            1. Coherence (logical flow and clarity)
            2. Completeness (addresses all aspects of query)
            3. Accuracy (nutritionally sound advice)
            
            Query: {test_query}
            Response: {llm_response[:500]}
            
            Provide scores in format: Coherence: X/10, Completeness: Y/10, Accuracy: Z/10
            """
            
            eval_response = session.sql(f"""
                SELECT SNOWFLAKE.CORTEX.COMPLETE(
                    'mistral-7b',
                    '{eval_prompt.replace("'", "''")}'
                ) as EVALUATION
            """).collect()[0]['EVALUATION']
            
            # Parse scores (simple extraction)
            import re
            scores = re.findall(r'(\d+)/10', eval_response)
            if len(scores) >= 3:
                comparative_scores[model_name]['coherence'].append(int(scores[0]))
                comparative_scores[model_name]['completeness'].append(int(scores[1]))
                comparative_scores[model_name]['accuracy'].append(int(scores[2]))
                
                print(f"   • Coherence: {scores[0]}/10")
                print(f"   • Completeness: {scores[1]}/10")
                print(f"   • Accuracy: {scores[2]}/10")
            
        except Exception as e:
            print(f"   ⚠️ Error in evaluation: {str(e)}")

# Calculate averages
for model in ['snowflake', 'openai']:
    if comparative_scores[model]['coherence']:
        avg_coherence = np.mean(comparative_scores[model]['coherence'])
        avg_completeness = np.mean(comparative_scores[model]['completeness'])
        avg_accuracy = np.mean(comparative_scores[model]['accuracy'])
        
        llm_metrics[f'{model}_quality'] = {
            'coherence_score': float(avg_coherence),
            'completeness_score': float(avg_completeness),
            'accuracy_score': float(avg_accuracy),
            'overall_quality': float((avg_coherence + avg_completeness + avg_accuracy) / 3)
        }


In [ ]:
print("\n4️⃣ HALLUCINATION & FACTUAL GROUNDING TEST")
print("-"*50)

hallucination_test_queries = [
    {
        'query': 'What is the protein content of chicken breast?',
        'verify_field': 'PROTEIN',
        'food_name_pattern': '%chicken%breast%'
    },
    {
        'query': 'How many calories are in an apple?',
        'verify_field': 'CALORIES',
        'food_name_pattern': '%apple%'
    }
]

factual_accuracy_scores = {'snowflake': [], 'openai': []}

for test_item in hallucination_test_queries:
    print(f"\n📝 Factual Query: '{test_item['query']}'")
    
    # Get ground truth from database
    ground_truth = session.sql(f"""
        SELECT 
            FOOD_NAME,
            {test_item['verify_field']} as VALUE
        FROM {source_table}
        WHERE LOWER(FOOD_NAME) LIKE LOWER('{test_item['food_name_pattern']}')
        LIMIT 5
    """).to_pandas()
    
    if not ground_truth.empty:
        true_values = ground_truth['VALUE'].dropna().tolist()
        avg_true_value = np.mean(true_values)
        
        print(f"   📌 Ground Truth: {avg_true_value:.2f} (avg from {len(true_values)} items)")
        
        for model in ['snowflake', 'openai']:
            if model == 'openai' and client is None:
                continue
                
            try:
                # Generate response using RAG
                if model == 'snowflake':
                    context_query = f"""
                        WITH query_embed AS (
                            SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                                'snowflake-arctic-embed-m', 
                                '{test_item['query'].replace("'", "''")}' 
                            ) as QUERY_EMBEDDING
                        )
                        SELECT 
                            s.FOOD_NAME,
                            m.{test_item['verify_field']} as VALUE
                        FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s
                        JOIN {source_table} m ON s.FDC_ID = m.FDC_ID,
                             query_embed q
                        ORDER BY VECTOR_COSINE_SIMILARITY(s.EMBEDDING, q.QUERY_EMBEDDING) DESC
                        LIMIT 3
                    """
                else:  # OpenAI
                    response = client.embeddings.create(
                        model="text-embedding-3-small",
                        input=test_item['query']
                    )
                    query_embedding = response.data[0].embedding
                    embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
                    
                    context_query = f"""
                        SELECT 
                            o.FOOD_NAME,
                            m.{test_item['verify_field']} as VALUE
                        FROM {embeddings_schema}.OPENAI_EMBEDDINGS o
                        JOIN {source_table} m ON o.FDC_ID = m.FDC_ID
                        ORDER BY VECTOR_COSINE_SIMILARITY(
                            o.EMBEDDING,
                            {embedding_str}::VECTOR(FLOAT, 1536)
                        ) DESC
                        LIMIT 3
                    """
                
                retrieved_data = session.sql(context_query).to_pandas()
                
                # Create factual context
                context = "\n".join([
                    f"- {row['FOOD_NAME']}: {row['VALUE']:.2f}" 
                    for _, row in retrieved_data.iterrows()
                ])
                
                # Generate response
                llm_response = session.sql(f"""
                    SELECT SNOWFLAKE.CORTEX.COMPLETE(
                        'mistral-7b',
                        'Based on our database:
                        {context.replace("'", "''")}
                        
                        Question: {test_item['query'].replace("'", "''")}
                        
                        Provide a specific numerical answer based on the data above.'
                    ) as RESPONSE
                """).collect()[0]['RESPONSE']
                
                # Extract number from response
                numbers = re.findall(r'\d+\.?\d*', llm_response)
                if numbers:
                    predicted_value = float(numbers[0])
                    error_rate = abs(predicted_value - avg_true_value) / avg_true_value
                    accuracy = max(0, 1 - error_rate)
                    
                    factual_accuracy_scores[model].append(accuracy)
                    
                    print(f"   {model.upper()}: Predicted {predicted_value:.2f}, Accuracy: {accuracy:.2%}")
                
            except Exception as e:
                print(f"   ⚠️ Error for {model}: {str(e)}")

# Store factual accuracy metrics
for model in ['snowflake', 'openai']:
    if factual_accuracy_scores[model]:
        llm_metrics[f'{model}_factual'] = {
            'avg_factual_accuracy': float(np.mean(factual_accuracy_scores[model])),
            'min_accuracy': float(np.min(factual_accuracy_scores[model])),
            'max_accuracy': float(np.max(factual_accuracy_scores[model]))
        }


In [ ]:
print("\n5️⃣ CONTEXT WINDOW UTILIZATION ANALYSIS")
print("-"*50)

context_sizes = [3, 5, 10, 20]
context_performance = {'snowflake': {}, 'openai': {}}

test_query = "Create a comprehensive weekly meal plan for weight loss"

for context_size in context_sizes:
    print(f"\n📊 Testing with context size: {context_size} items")
    
    for model in ['snowflake', 'openai']:
        if model == 'openai' and client is None:
            continue
            
        try:
            # Retrieve different amounts of context
            if model == 'snowflake':
                results = session.sql(f"""
                    WITH query_embed AS (
                        SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                            'snowflake-arctic-embed-m', 
                            '{test_query.replace("'", "''")}' 
                        ) as QUERY_EMBEDDING
                    )
                    SELECT 
                        s.FOOD_NAME,
                        s.CATEGORY,
                        VECTOR_COSINE_SIMILARITY(s.EMBEDDING, q.QUERY_EMBEDDING) as SIMILARITY
                    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s, query_embed q
                    ORDER BY SIMILARITY DESC
                    LIMIT {context_size}
                """).to_pandas()
            else:  # OpenAI
                response = client.embeddings.create(
                    model="text-embedding-3-small",
                    input=test_query
                )
                query_embedding = response.data[0].embedding
                embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
                
                results = session.sql(f"""
                    SELECT 
                        FOOD_NAME,
                        CATEGORY,
                        VECTOR_COSINE_SIMILARITY(
                            EMBEDDING,
                            {embedding_str}::VECTOR(FLOAT, 1536)
                        ) as SIMILARITY
                    FROM {embeddings_schema}.OPENAI_EMBEDDINGS
                    ORDER BY SIMILARITY DESC
                    LIMIT {context_size}
                """).to_pandas()
            
            # Measure response quality with different context sizes
            avg_similarity = results['SIMILARITY'].mean()
            diversity = len(results['CATEGORY'].dropna().unique()) / len(results)
            
            context_performance[model][context_size] = {
                'avg_similarity': float(avg_similarity),
                'category_diversity': float(diversity),
                'items_retrieved': len(results)
            }
            
            print(f"   {model.upper()}: Similarity={avg_similarity:.4f}, Diversity={diversity:.2f}")
            
        except Exception as e:
            print(f"   ⚠️ Error for {model}: {str(e)}")

# Determine optimal context size
for model in ['snowflake', 'openai']:
    if context_performance[model]:
        # Find context size with best balance of relevance and diversity
        scores = []
        for size, metrics in context_performance[model].items():
            score = metrics['avg_similarity'] * 0.7 + metrics['category_diversity'] * 0.3
            scores.append((size, score))
        
        optimal_size = max(scores, key=lambda x: x[1])
        llm_metrics[f'{model}_context_optimization'] = {
            'optimal_context_size': optimal_size[0],
            'optimal_score': float(optimal_size[1])
        }
        
        print(f"\n   {model.upper()} Optimal Context Size: {optimal_size[0]} items")


In [ ]:
print("\n6️⃣ MULTI-TURN CONVERSATION COHERENCE")
print("-"*50)

conversation_flow = [
    "I want to start eating healthier",
    "What are some high-protein options?",
    "Which of those are vegetarian?",
    "Can you suggest a meal plan with those?"
]

conversation_coherence = {'snowflake': [], 'openai': []}
conversation_context = {'snowflake': [], 'openai': []}

for model in ['snowflake', 'openai']:
    if model == 'openai' and client is None:
        continue
        
    print(f"\n{model.upper()} Multi-turn Test:")
    accumulated_context = []
    
    for turn_idx, query in enumerate(conversation_flow[:3]):  # Test first 3 turns
        print(f"\n   Turn {turn_idx + 1}: '{query}'")
        
        try:
            # Build cumulative query
            full_query = " ".join(conversation_flow[:turn_idx + 1])
            
            # Retrieve relevant context
            if model == 'snowflake':
                results = session.sql(f"""
                    WITH query_embed AS (
                        SELECT SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                            'snowflake-arctic-embed-m', 
                            '{full_query.replace("'", "''")}' 
                        ) as QUERY_EMBEDDING
                    )
                    SELECT 
                        FOOD_NAME,
                        CATEGORY,
                        VECTOR_COSINE_SIMILARITY(EMBEDDING, QUERY_EMBEDDING) as SIMILARITY
                    FROM {embeddings_schema}.SNOWFLAKE_EMBEDDINGS s, query_embed q
                    ORDER BY SIMILARITY DESC
                    LIMIT 3
                """).to_pandas()
            else:  # OpenAI
                response = client.embeddings.create(
                    model="text-embedding-3-small",
                    input=full_query
                )
                query_embedding = response.data[0].embedding
                embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'
                
                results = session.sql(f"""
                    SELECT 
                        FOOD_NAME,
                        CATEGORY,
                        VECTOR_COSINE_SIMILARITY(
                            EMBEDDING,
                            {embedding_str}::VECTOR(FLOAT, 1536)
                        ) as SIMILARITY
                    FROM {embeddings_schema}.OPENAI_EMBEDDINGS
                    ORDER BY SIMILARITY DESC
                    LIMIT 3
                """).to_pandas()
            
            # Check coherence: are results building on previous context?
            current_foods = set(results['FOOD_NAME'].tolist())
            
            if accumulated_context:
                # Check if maintaining context
                overlap = len(current_foods.intersection(set(accumulated_context))) / len(current_foods)
                conversation_coherence[model].append(overlap)
                print(f"   Context retention: {overlap:.2%}")
            
            accumulated_context.extend(current_foods)
            
        except Exception as e:
            print(f"   ⚠️ Error: {str(e)}")
    
    if conversation_coherence[model]:
        llm_metrics[f'{model}_conversation'] = {
            'avg_context_retention': float(np.mean(conversation_coherence[model])),
            'conversation_coherence_score': float(1 - np.std(conversation_coherence[model]))
        }

In [ ]:
print("\n" + "="*70)
print("🤖 LLM-BASED EVALUATION SUMMARY")
print("="*70)

print("\n🏆 RAG PIPELINE PERFORMANCE:")
print("-"*50)

if 'snowflake_rag' in llm_metrics and 'openai_rag' in llm_metrics:
    print("📊 Snowflake Arctic + LLM:")
    for metric, value in llm_metrics['snowflake_rag'].items():
        print(f"   • {metric.replace('_', ' ').title()}: {value:.3f}")
    
    print("\n📊 OpenAI + LLM:")
    for metric, value in llm_metrics['openai_rag'].items():
        print(f"   • {metric.replace('_', ' ').title()}: {value:.3f}")
    
    # Compare performance
    snow_score = llm_metrics['snowflake_rag']['avg_relevance'] * llm_metrics['snowflake_rag']['avg_keyword_coverage']
    openai_score = llm_metrics['openai_rag']['avg_relevance'] * llm_metrics['openai_rag']['avg_keyword_coverage']
    
    print(f"\n🎯 Overall RAG Quality Score:")
    print(f"   • Snowflake: {snow_score:.4f}")
    print(f"   • OpenAI: {openai_score:.4f}")
    print(f"   • Winner: {'Snowflake' if snow_score > openai_score else 'OpenAI'}")

print("\n📈 RESPONSE QUALITY METRICS:")
print("-"*50)

if 'snowflake_quality' in llm_metrics:
    print("Snowflake Pipeline:")
    print(f"   • Overall Quality: {llm_metrics['snowflake_quality']['overall_quality']:.1f}/10")

if 'openai_quality' in llm_metrics:
    print("OpenAI Pipeline:")
    print(f"   • Overall Quality: {llm_metrics['openai_quality']['overall_quality']:.1f}/10")

print("\n✅ FACTUAL ACCURACY:")
print("-"*50)

if 'snowflake_factual' in llm_metrics:
    print(f"Snowflake: {llm_metrics['snowflake_factual']['avg_factual_accuracy']:.2%} accurate")

if 'openai_factual' in llm_metrics:
    print(f"OpenAI: {llm_metrics['openai_factual']['avg_factual_accuracy']:.2%} accurate")
